
<div class="alert alert-dark" role="alert">

## Function Declaration

#### Data analysis of DDB_PD_138_AMBR and DDB_PD_139_AMBR experiments:

Author: 
`Haroun Bensaadi`

**Version: 23.01.2024**
</div>



In [2]:
# pip freeze > requirements.txt

In [3]:
%%capture
pip install -r requirements.txt

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from colorama import init, Fore, Style
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import MaxNLocator, FuncFormatter
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
from chempy import Substance
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display, HTML
import ipywidgets as widgets
import os
import platform
import subprocess

pd.set_option('display.precision', 6)
pd.set_option('display.float_format', '{:.6f}'.format)
pd.set_option('display.max_columns', 100)

files = ["AMBR_data.xlsx", "elemental_composition_media.xlsx", "elemental_composition_compounds.xlsx", "OD_measurements.xlsx", "TM_supernatent.xlsx", "TM_biomass.xlsx", "dry_weight_final_sample.xlsx"]

atoms = ["C","H","O","N","S","P","Cl","Na","K","Mg","Ca","Fe","Mn","Zn","Cu","Co","Mo"]

media_per_reactor = {
    '138_R01': ['DDB_BM_006', 'DDB_FM_006'],
    '138_R02': ['DDB_BM_006', 'DDB_FM_006'],
    '138_R03': ['DDB_BM_006', 'DDB_FM_006'],
    '138_R04': ['DDB_BM_003', 'DDB_FM_003'],
    '138_R05': ['DDB_BM_003', 'DDB_FM_003'],
    '138_R06': ['DDB_BM_003', 'DDB_FM_003'],
    '139_R06': ['DDB_BM_006', 'DDB_FM_006'],
    '139_R07': ['DDB_BM_006', 'DDB_FM_006'],
    '139_R08': ['DDB_BM_006', 'DDB_FM_006'],
    '139_R09': ['DDB_BM_003', 'DDB_FM_003'],
    '139_R10': ['DDB_BM_003', 'DDB_FM_003'],
    '139_R11': ['DDB_BM_003', 'DDB_FM_003'],
}

LODs = {"Mn" : "50 µg/L", 
        "Fe" : "100 µg/L",
        "Co" : "50 µg/L",
        "Cu" : "100 µg/L",
        "Zn" : "100 µg/L",
        "Mo" : "100 µg/L",
        "Na" : "500 mg/L",
        "Mg" : "10 mg/L",
        "K"  : "500 mg/L",
        "Ca" : "10 mg/L"}

main_elements = ["C", "H", "O"]
TM_elements = ["Fe", "Zn", "Mg", "Mn", "Cu", "Co", "K", "Mo"]

FOLDER_NAME_1 = "1-Metabolites Evolution [mol + L] vs [h]"
FOLDER_NAME_2 = "2-Metabolite Evolution [g] vs [h]"
FOLDER_NAME_3 = "3-Metabolite Evolution [mol] vs [h]"
FOLDER_NAME_4 = "4-Consumption and Production Rates [g.L-1.h-1 or mol.L-1.h-1] vs [h]"
FOLDER_NAME_5 = "5-Biomass-Specific Consumption and Production Rates [g.g-1.h-1] or [mol.mol-1.h-1] vs [h]"
FOLDER_NAME_6 = "6-Substrate and Product Yields [g.g-1] or [mol.mol-1] vs [h]"

FOLDER_NAME_7 = "7-Mass Balance Main Elements [mmol] vs [h]"
FOLDER_NAME_8 = "8-Mass Balance Main Elements [%] vs [h]"
FOLDER_NAME_9 = "9-Mass Balance TM Elements [mmol] vs [h]"
FOLDER_NAME_10 = "10-Mass Balance TM Elements [%] vs [h]"

FOLDER_NAME_11 = "11-Elemental Growth Yield Slopes [g biomass] vs [g element]"
FOLDER_NAME_12 = "12-Elemental Growth Yield Against Reference Values"

FOLDER_NAME_13 = "13-Yield Slopes [µg element] vs [g biomass]"
FOLDER_NAME_14 = "14-Yield Bar Plots [µg element.g biomass-1]"
FOLDER_NAME_15 = "15-Yield Bar Plot [avg. fed-batch µg element.g biomass-1] vs per medium and strain type"
FOLDER_NAME_16 = "16-Yield Bar Plot [avg. fed-batch µg element.g biomass-1] vs per strain type"

FOLDER_NAME_17 = "17-Yield Extra Plots"

IS_SAVE_FIGURES = True
IS_SAVE_TABLES = False

AMBR_data_dfs = {}
organized_AMBR_data_dfs = {}
mass_balance_data_per_elements = {}
final_massic_concentration_per_element = {}
mass_balance_dfs = {}
mass_balance_TM_elements_dfs = {}
elemental_growth_yields_per_reactor = {}
yields_per_reactor = {}
yields_per_reactor_df = pd.DataFrame()
intercept_per_reactor_df = pd.DataFrame()

NUMBER_OF_REACTORS = 12

##  Declared Functions

In [5]:
### helping functions

def createFolderInsideFiguresFolder():
    global FOLDER_NAME_1, FOLDER_NAME_2, FOLDER_NAME_3, FOLDER_NAME_4, FOLDER_NAME_5, FOLDER_NAME_6, FOLDER_NAME_7, FOLDER_NAME_8
    global FOLDER_NAME_9, FOLDER_NAME_10, FOLDER_NAME_11, FOLDER_NAME_12, FOLDER_NAME_13, FOLDER_NAME_14, FOLDER_NAME_15, FOLDER_NAME_16 
    global FOLDER_NAME_17
    
    file_path = "figures/"

    folder_names = [FOLDER_NAME_1, FOLDER_NAME_2, FOLDER_NAME_3, FOLDER_NAME_4, FOLDER_NAME_5, FOLDER_NAME_6, FOLDER_NAME_7, FOLDER_NAME_8, 
                    FOLDER_NAME_9, FOLDER_NAME_10, FOLDER_NAME_11, FOLDER_NAME_12, FOLDER_NAME_13, FOLDER_NAME_14, FOLDER_NAME_15, FOLDER_NAME_16, 
                    FOLDER_NAME_17]

    for folder_name in folder_names:
        folder_path = os.path.join(file_path, folder_name)
    
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            
def printText(text1, color1=Fore.WHITE, text2="", color2=Fore.WHITE, style=Style.BRIGHT):
    """
    Used to print text with colors
    """
    print(style + color1 + text1 + color2 + text2 + Style.RESET_ALL)

def createAndDisplayFolderButtons(folder_names):
    def openFolderInFileExplorer(folder_name):
        current_working_directory = os.getcwd()
        folder_path = os.path.join(current_working_directory, f"figures/{folder_name}")
        
        if platform.system() == "Windows":
            os.startfile(folder_path, 'explore')
        elif platform.system() == "Darwin":
            subprocess.run(["open", "-R", folder_path])

    display(HTML("""
    <style>
        .widget-label { font-size: 500%; vertical-align: middle; line-height: normal; }
        .fa-folder { font-size: 250%; vertical-align: middle; margin-left: 0px; }
        .fa-folder:before { color: #ffda73 !important; }
    </style>
    """))

    message_label = widgets.HTML(value="<strong style='font-size: 16px; color: rgba(20, 20, 20, 0.60);'>The generated images are located inside the folders: </strong>")            

    buttons = []
    for folder_name in folder_names:
        button = widgets.Button(description=folder_name, 
                                icon='folder', 
                                layout=widgets.Layout(width='auto', height="auto", margin='5px 0px', padding='7.5px 15px'), 
                                style={'button_color': 'rgba(211, 211, 211, 0.35)'})
        button.on_click(lambda b, folder_name=folder_name: openFolderInFileExplorer(folder_name))
        buttons.append(button)
    
    display(widgets.VBox([message_label, *buttons], layout=widgets.Layout(align_items='flex-start')))

def fillGaps(data, column_name):
    """
    Used to fill the gaps in the column "Liquid volume [L]" resulting from sampling so the plot looks nice
    
    """
    data[column_name] = data[column_name].interpolate()

def findStartFeed(reactor_data):
    start_feed_index = None
    for i, value in enumerate(reactor_data["Feed 1 volume [L]"]):
        if not pd.isna(value) and value != 0:
            start_feed_index = i
            break
    return reactor_data.loc[start_feed_index, "Process Time [h]"]

def getTitleInformation(reactor_name, bacth_medium, feed_medium):
    if "006" in bacth_medium and "006" in feed_medium:
        iron_zinc_condition = "↑ Zn+Fe"
    elif "003" in bacth_medium and "003" in feed_medium:
        iron_zinc_condition = "↓ Zn+Fe"

    if reactor_name in ["138_R01", "138_R02", "138_R03", "138_R04", "138_R05", "138_R06"]:
        used_strain = "HMP3071"
    elif reactor_name in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
        used_strain = "DDB35"

    return iron_zinc_condition, used_strain

def getMolarMass(formula):
    return round(Substance.from_formula(formula).mass, 5)

def derivative(x, t):
    
    derivative = [0] * len(t)  

    for i in range(len(t)):
        if i == 0 or i == len(t) - 1:
            derivative[i] = np.nan
        elif i == 1:
            derivative[i] = (x[i + 1] - x[i]) / (t[i + 1] - t[i])
        elif i == len(t) - 2:
            derivative[i] = (x[i] - x[i - 1]) / (t[i] - t[i - 1])
        else:
            derivative[i] = (x[i + 1] - x[i - 1]) / (t[i + 1] - t[i - 1])

    return derivative

def openXlsxFiles():
    global AMBR_data_dfs
    global elemental_composition_media_df, elemental_composition_compounds_df, manual_OD_measurements_df, TM_supernatent_df, TM_biomass_df, dry_weight_final_sample_df

    AMBR_data_sheet_names = list(pd.read_excel(f"data/{files[0]}", sheet_name=None).keys())
    AMBR_data_sheet_names.remove('Feature table')

    for AMBR_data_sheet_name in AMBR_data_sheet_names:
        AMBR_data_dfs[AMBR_data_sheet_name] = pd.read_excel(f"data/{files[0]}", sheet_name=AMBR_data_sheet_name)
        
    elemental_composition_media_df = pd.read_excel(f"data/{files[1]}", sheet_name="g ELEMENT per L MEDIA")
    elemental_composition_compounds_df = pd.read_excel(f"data/{files[2]}", sheet_name="g ELEMENT per g COMPOUND")
    manual_OD_measurements_df = pd.read_excel(f"data/{files[3]}", sheet_name="data")
    TM_supernatent_df = pd.read_excel(f"data/{files[4]}", sheet_name="data_without_rsd")
    TM_biomass_df = pd.read_excel(f"data/{files[5]}", sheet_name="data_without_rsd")
    dry_weight_final_sample_df = pd.read_excel(f"data/{files[6]}", sheet_name="data")

def printAMBRtables(number_of_reactors = 12):
    global organized_AMBR_data_dfs

    for reactor_name, organized_AMBR_data_df in list(organized_AMBR_data_dfs.items())[:number_of_reactors]:
        printText("Reactor: ", Fore.BLACK, f"{reactor_name}", Fore.BLUE)
        
        display(organized_AMBR_data_df)
        
        if IS_SAVE_TABLES:
            organized_AMBR_data_df.to_excel(f'tables/organized_data_{reactor_name}.xlsx', index=False)


#### Functions for generating figures

1-Metabolites Evolution [mol + L] vs [h]

2-Metabolite Evolution [g] vs [h]

3-Metabolite Evolution [mol] vs [h]


In [6]:

def plotMetaboliteEvolutionAMBRdata(number_of_reactors=12, folder_name=FOLDER_NAME_1):
    global organized_AMBR_data_dfs
    
    for i in range(0, number_of_reactors, 3):
        reactor_names  = list(AMBR_data_dfs.keys())[i:i+3]
        
        fillGaps(AMBR_data_dfs[reactor_names[0]], "Liquid volume [L]")
        fillGaps(AMBR_data_dfs[reactor_names[1]], "Liquid volume [L]")
        fillGaps(AMBR_data_dfs[reactor_names[2]], "Liquid volume [L]")
        
        start_feed_time_r_1 = findStartFeed(AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(AMBR_data_dfs[reactor_names[2]])
        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3

        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)
        
        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
  
        fig, ax1 = plt.subplots(figsize=(8*0.90, 5*0.90))
        alpha_value = 0.50
        
        ax1.axvline(x=0, color='black', linestyle='--', alpha=0.30)
        ax2 = ax1.twinx()

        avg_values = {}
        std_values = {}

        for column in ['Process Time [h]', 'Liquid volume [L]', 'Feed 1 volume [L]','Base volume [L]', 'CTR [mol/h]', 'Consumed O2 [mol]', 'Formed CO2 [mol]']:
            avg_values[column] = np.mean([organized_AMBR_data_dfs[reactor_names[0]][column], organized_AMBR_data_dfs[reactor_names[1]][column], organized_AMBR_data_dfs[reactor_names[2]][column]], axis=0)
            std_values[column] = np.std([organized_AMBR_data_dfs[reactor_names[0]][column], organized_AMBR_data_dfs[reactor_names[1]][column], organized_AMBR_data_dfs[reactor_names[2]][column]], axis=0)

        ax1.plot(AMBR_data_dfs[reactor_names[0]]["Process Time [h]"]-start_feed_time_r_1, AMBR_data_dfs[reactor_names[0]]["Liquid volume [L]"], label=f'Liquid Volume [L]', linestyle='--', color='blue', alpha=alpha_value)
        ax1.plot(AMBR_data_dfs[reactor_names[1]]["Process Time [h]"]-start_feed_time_r_2, AMBR_data_dfs[reactor_names[1]]["Liquid volume [L]"], linestyle='--', color='blue', alpha=alpha_value)
        ax1.plot(AMBR_data_dfs[reactor_names[2]]["Process Time [h]"]-start_feed_time_r_3, AMBR_data_dfs[reactor_names[2]]["Liquid volume [L]"], linestyle='--', color='blue', alpha=alpha_value)

        ax2.plot(AMBR_data_dfs[reactor_names[0]]["Process Time [h]"]-start_feed_time_r_1, AMBR_data_dfs[reactor_names[0]]["Feed 1 volume [L]"], label=f'Feed 1 volume [L]', linestyle='-', color='red', alpha=alpha_value)
        ax2.plot(AMBR_data_dfs[reactor_names[1]]["Process Time [h]"]-start_feed_time_r_2, AMBR_data_dfs[reactor_names[1]]["Feed 1 volume [L]"], linestyle='-', color='red', alpha=alpha_value)
        ax2.plot(AMBR_data_dfs[reactor_names[2]]["Process Time [h]"]-start_feed_time_r_3, AMBR_data_dfs[reactor_names[2]]["Feed 1 volume [L]"], linestyle='-', color='red', alpha=alpha_value)
        
        ax2.plot(AMBR_data_dfs[reactor_names[0]]["Process Time [h]"]-start_feed_time_r_1, AMBR_data_dfs[reactor_names[0]]["Base volume [L]"], label=f'Base volume [L]', linestyle='-', color='orange', alpha=alpha_value)
        ax2.plot(AMBR_data_dfs[reactor_names[1]]["Process Time [h]"]-start_feed_time_r_2, AMBR_data_dfs[reactor_names[1]]["Base volume [L]"], linestyle='-', color='orange', alpha=alpha_value)
        ax2.plot(AMBR_data_dfs[reactor_names[2]]["Process Time [h]"]-start_feed_time_r_3, AMBR_data_dfs[reactor_names[2]]["Base volume [L]"], linestyle='-', color='orange', alpha=alpha_value)

        ax1.plot(AMBR_data_dfs[reactor_names[0]]["Process Time [h]"]-start_feed_time_r_1, AMBR_data_dfs[reactor_names[0]]["Consumed O2 [mol]"], label=f'Consumed O2 [mol]', linestyle='--', color='magenta', alpha=alpha_value)
        ax1.plot(AMBR_data_dfs[reactor_names[1]]["Process Time [h]"]-start_feed_time_r_2, AMBR_data_dfs[reactor_names[1]]["Consumed O2 [mol]"], linestyle='--', color='magenta', alpha=alpha_value)
        ax1.plot(AMBR_data_dfs[reactor_names[2]]["Process Time [h]"]-start_feed_time_r_3, AMBR_data_dfs[reactor_names[2]]["Consumed O2 [mol]"], linestyle='--', color='magenta', alpha=alpha_value)
        
        ax1.plot(AMBR_data_dfs[reactor_names[0]]["Process Time [h]"]-start_feed_time_r_1, AMBR_data_dfs[reactor_names[0]]["Formed CO2 [mol]"], label=f'Formed CO2 [mol]', linestyle='--', color='green', alpha=alpha_value)
        ax1.plot(AMBR_data_dfs[reactor_names[1]]["Process Time [h]"]-start_feed_time_r_2, AMBR_data_dfs[reactor_names[1]]["Formed CO2 [mol]"], linestyle='--', color='green', alpha=alpha_value)
        ax1.plot(AMBR_data_dfs[reactor_names[2]]["Process Time [h]"]-start_feed_time_r_3, AMBR_data_dfs[reactor_names[2]]["Formed CO2 [mol]"], linestyle='--', color='green', alpha=alpha_value)
        
        ax1.set_xlabel('Process Time [h]')

        ax1.set_ylabel('dashed line', color='black')
        ax2.set_ylabel('', color='black')
        
        ax2.set_ylim(0, 0.05)
        ax1.set_ylim(0, 0.25)

        ax1.tick_params(axis='y', labelcolor='black')
        ax2.tick_params(axis='y', labelcolor='black')
        
        ax1.text(0.23, 0.95, 'Batch', transform=ax1.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.75)
        ax1.text(0.70, 0.95, 'Fed-batch', transform=ax1.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.75)

        x_max = avg_values["Process Time [h]"].max()-avg_start_feed_time
        ax1.set_xlim(-avg_start_feed_time - ((x_max)*0.045), x_max + ((x_max)*0.045))

        plt.title(f'Reactors: {reactor_name_triplicate}    |    Strain: {used_strain}    |    Medium: {iron_zinc_condition}\n')
        
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines2 + lines, labels2 + labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/ambr_data_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotMetaboliteEvolutionMassPerTime(number_of_reactors=12, folder_name=FOLDER_NAME_2):
    global organized_AMBR_data_dfs

    for i in range(0, number_of_reactors, 3):
        reactor_names = list(organized_AMBR_data_dfs.keys())[i:i+3]

        start_feed_time_r_1 = findStartFeed(organized_AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(organized_AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(organized_AMBR_data_dfs[reactor_names[2]])

        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3

        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"

        fig, ax1 = plt.subplots(figsize=(8*0.85, 5*0.85))
        alpha_value = 0.50
        alpha_value_bars = 0.30

        ax1.set_xlabel('Process Time [h]')
        ax1.tick_params(axis='y', labelcolor='black')        
        
        ax1.axvline(x=0, color='black', linestyle='--', alpha=0.30)
        ax1.axhline(y=0, color='black', linestyle='-', alpha=0.45)
        
        ax2 = ax1.twinx()

        avg_values = {}
        std_values = {}

        for column in ['Process Time [h]', 'Consumed D-glucose [g]', 'acetic acid [g]','tryptophan [g]', 'Base [g]', 'Consumed O2 [g]', 'Formed CO2 [g]', 'Biomass [g]']:
            avg_values[column] = np.mean([organized_AMBR_data_dfs[reactor_names[0]][column], organized_AMBR_data_dfs[reactor_names[1]][column], organized_AMBR_data_dfs[reactor_names[2]][column]], axis=0)
            std_values[column] = np.std([organized_AMBR_data_dfs[reactor_names[0]][column], organized_AMBR_data_dfs[reactor_names[1]][column], organized_AMBR_data_dfs[reactor_names[2]][column]], axis=0)

        ax2.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Consumed D-glucose [g]'], yerr=std_values['Consumed D-glucose [g]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)                
        ax2.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Consumed D-glucose [g]'], marker='s', linestyle='-', label=f'Consumed D-glucose [g]', color='blue',  alpha=alpha_value)
        
        ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['acetic acid [g]'], yerr=std_values['acetic acid [g]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['acetic acid [g]'], marker='D', linestyle='--', label=f'acetic acid [g]', color='orange',  alpha=alpha_value)
        
        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['tryptophan [g]'], yerr=std_values['tryptophan [g]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
            ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'tryptophan [g]'], marker='P', linestyle='--', label=f'tryptophan [g]', color='magenta',  alpha=alpha_value)
            
        ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Base [g]'], yerr=std_values['Base [g]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Base [g]'], marker='v', linestyle='--', label=f'Base [g]', color='purple',  alpha=alpha_value)
        
        ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Consumed O2 [g]'], yerr=std_values['Consumed O2 [g]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Consumed O2 [g]'], marker='p', linestyle='--', label=f'Consumed O2 [g]', color='red',  alpha=alpha_value)
        
        ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Formed CO2 [g]'], yerr=std_values['Formed CO2 [g]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Formed CO2 [g]'], marker='^', linestyle='--', label=f'Formed CO2 [g]', color='brown',  alpha=alpha_value)
        
        ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Biomass [g]'], yerr=std_values['Biomass [g]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Biomass [g]'], marker='h', linestyle='--', label=f'Biomass [g]', color='green',  alpha=alpha_value)
        
        ax2.tick_params(axis='y', labelcolor='black')
        
        x_max = avg_values['Process Time [h]'].max()-avg_start_feed_time
        ax1.set_xlim(-avg_start_feed_time - ((x_max)*0.05), x_max + ((x_max)*0.05))

        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines2 + lines, labels2 + labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

        ax1.set_ylabel('mass [g] dashed line', color='black')
        ax2.set_ylabel('mass [g]', color='black')

        ax1.text(0.23, 0.95, 'Batch', transform=ax1.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.75)
        ax1.text(0.70, 0.95, 'Fed-batch', transform=ax1.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.75)

        plt.title(f'{reactor_name_triplicate}    |    {used_strain}    |    {iron_zinc_condition}')
        
        ax1.set_ylim(-(12.5*0.05), 12.5)
        ax2.set_ylim(-(35*0.05), 35)
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/mass_plot_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotMetaboliteEvolutionQuantityPerTime(number_of_reactors=12, folder_name=FOLDER_NAME_3):
    global organized_AMBR_data_dfs
    for i in range(0, number_of_reactors, 3):
        reactor_names = list(organized_AMBR_data_dfs.keys())[i:i+3]
            
        start_feed_time_r_1 = findStartFeed(organized_AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(organized_AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(organized_AMBR_data_dfs[reactor_names[2]])

        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3

        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)
        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"

        fig, ax1 = plt.subplots(figsize=(8*0.85, 5*0.85))
        alpha_value = 0.50
        alpha_value_bars = 0.30

        ax1.set_xlabel('Process Time [h]')
        ax1.tick_params(axis='y', labelcolor='black')        
        
        ax1.axvline(x=0, color='black', linestyle='--', alpha=0.30)
        ax1.axhline(y=0, color='black', linestyle='-', alpha=0.45)
        
        ax2 = ax1.twinx()

        avg_values = {}
        std_values = {}

        for column in ['Process Time [h]', 'Consumed D-glucose [mmol]', 'acetic acid [mmol]','tryptophan [mmol]', 'Base [mmol]', 'Consumed O2 [mmol]', 'Formed CO2 [mmol]', 'Biomass [mmol]']:
            avg_values[column] = np.mean([organized_AMBR_data_dfs[reactor_names[0]][column], organized_AMBR_data_dfs[reactor_names[1]][column], organized_AMBR_data_dfs[reactor_names[2]][column]], axis=0)
            std_values[column] = np.std([organized_AMBR_data_dfs[reactor_names[0]][column], organized_AMBR_data_dfs[reactor_names[1]][column], organized_AMBR_data_dfs[reactor_names[2]][column]], axis=0)
                        
        
        ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['acetic acid [mmol]'], yerr=std_values['acetic acid [mmol]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['acetic acid [mmol]'], marker='D', linestyle='--', label=f'acetic acid [mmol]', color='orange',  alpha=alpha_value)
        
        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            ax1.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['tryptophan [mmol]'], yerr=std_values['tryptophan [mmol]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
            ax1.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'tryptophan [mmol]'], marker='P', linestyle='--', label=f'tryptophan [mmol]', color='magenta',  alpha=alpha_value)

        ax2.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Consumed D-glucose [mmol]'], yerr=std_values['Consumed D-glucose [mmol]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax2.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Consumed D-glucose [mmol]'], marker='s', linestyle='-', label=f'Consumed D-glucose [mmol]', color='blue',  alpha=alpha_value)
        
        ax2.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Base [mmol]'], yerr=std_values['Base [mmol]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax2.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Base [mmol]'], marker='v', linestyle='-', label=f'Base [mmol]', color='purple',  alpha=alpha_value)
        
        ax2.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Consumed O2 [mmol]'], yerr=std_values['Consumed O2 [mmol]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax2.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Consumed O2 [mmol]'], marker='p', linestyle='-', label=f'Consumed O2 [mmol]', color='red',  alpha=alpha_value)
        
        ax2.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Formed CO2 [mmol]'], yerr=std_values['Formed CO2 [mmol]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax2.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Formed CO2 [mmol]'], marker='^', linestyle='-', label=f'Formed CO2 [mmol]', color='brown',  alpha=alpha_value)
        
        ax2.errorbar(avg_values['Process Time [h]']-avg_start_feed_time, avg_values['Biomass [mmol]'], yerr=std_values['Biomass [mmol]'], color='black', capsize=7, linestyle="", alpha=alpha_value_bars)
        ax2.plot(avg_values['Process Time [h]']-avg_start_feed_time, avg_values[f'Biomass [mmol]'], marker='h', linestyle='-', label=f'Biomass [mmol]', color='green',  alpha=alpha_value)
        
        ax2.tick_params(axis='y', labelcolor='black')
        
        x_max = avg_values['Process Time [h]'].max()-avg_start_feed_time
        ax1.set_xlim(-avg_start_feed_time - ((x_max)*0.05), x_max + ((x_max)*0.05))

        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines2+lines, labels2+labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

        ax1.set_ylabel('quantity [mmol] dashed line', color='black')
        ax2.set_ylabel('quantity [mmol]', color='black')

        ax1.set_ylim(-(3.25*0.05), 3.25)
        ax2.set_ylim(-(325*0.05), 325)
        
        ax1.text(0.23, 0.95, 'Batch', transform=ax1.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.75)
        ax1.text(0.70, 0.95, 'Fed-batch', transform=ax1.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.75)

        plt.title(f'{reactor_name_triplicate}    |    {used_strain}    |    {iron_zinc_condition}')
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/quantity_plot_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')
        
        plt.show()


#### Functions for generating figures

4-Consumption and Production Rates [g.L-1.h-1 or mol.L-1.h-1] vs [h]

5-Biomass-Specific Consumption and Production Rates [g.g-1.h-1] or [mol.mol-1.h-1] vs [h]

6-Substrate and Product Yields [g.g-1] or [mol.mol-1] vs [h]

In [7]:
def calcuateRatesAndSubstrateAndProductYields(number_of_reactors=12):
    global organized_AMBR_data_dfs, rates_and_yields_dfs

    rates_and_yields_dfs = {}

    for reactor_name, AMBR_data_df in list(organized_AMBR_data_dfs.items())[:number_of_reactors]:
        rates_and_yields = {
            'Process Time [h]': [],

            'R glucose [g/L/h]': [],
            'R glucose [mol/L/h]': [],
            'q glucose [g/g/h]': [],
            'q glucose [mol/mol/h]': [],
            
            'R O2 [g/L/h]': [],
            'R O2 [mol/L/h]': [],
            'q O2 [g/g/h]': [],
            'q O2 [mol/mol/h]': [],
            
            'R base [g/L/h]': [],
            'R base [mol/L/h]': [],
            'q base [g/g/h]': [],
            'q base [mol/mol/h]': [],
            
            'R biomass [g/L/h]': [],
            'R biomass [mol/L/h]': [],
            'q biomass (µ) [1/h]': [],

            'R CO2 [g/L/h]': [],
            'R CO2 [mol/L/h]': [],
            'q CO2 [g/g/h]': [],
            'q CO2 [mol/mol/h]': [],
            
            'R acetic acid [g/L/h]': [],
            'R acetic acid [mol/L/h]': [],
            'q acetic acid [g/g/h]': [],
            'q acetic acid [mol/mol/h]': [],
            
            'R tryptophan [g/L/h]': [],
            'R tryptophan [mol/L/h]': [],
            'q tryptophan [g/g/h]': [],
            'q tryptophan [mol/mol/h]': [],

            'Y biomass [g/g]': [],
            'Y biomass [mol/mol]': [],
            'Y tryptophan [g/g]': [],
            'Y tryptophan [mol/mol]': [],
            'Y acetic acid [g/g]': [],
            'Y acetic acid [mol/mol]': [],
            'Y CO2 [g/g]': [],
            'Y CO2 [mol/mol]': [],

        }

        t = AMBR_data_df['Process Time [h]']
        rates_and_yields['Process Time [h]'].extend(t)

        ### Concentrations
        # [g/L]
        Cm_glucose = AMBR_data_df['Consumed D-glucose [g]']/AMBR_data_df['Liquid volume [L]']
        Cm_O2 = AMBR_data_df['Consumed O2 [g/L]']     
        Cm_base = AMBR_data_df['Base [g/L]']            
        Cm_biomass = AMBR_data_df['Biomass [g/L]']      
        Cm_CO2 = AMBR_data_df['Formed CO2 [g/L]']
        Cm_acetic_acid = AMBR_data_df['acetic acid [g/L]']
        
        if reactor_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            Cm_tryptophan = AMBR_data_df['tryptophan [g/L]']

        # [mol/L]
        C_glucose = 1e-3*AMBR_data_df['Consumed D-glucose [mmol]']/AMBR_data_df['Liquid volume [L]']         
        C_O2 = AMBR_data_df['Consumed O2 [mol/L]']             
        C_base = AMBR_data_df['Base [mol/L]']                  
        C_biomass = AMBR_data_df['Biomass [mol/L]']           
        C_CO2 = AMBR_data_df['Formed CO2 [mol/L]']          
        C_acetic_acid = AMBR_data_df['acetic acid [mol/L]']   
        
        if reactor_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            C_tryptophan = AMBR_data_df['tryptophan [mol/L]'] 

        ### Rates
        # [g * L^-1 * h^-1]
        Rm_glucose = derivative(Cm_glucose, t)
        Rm_O2 = derivative(Cm_O2, t)
        Rm_base = derivative(Cm_base, t)
        Rm_biomass = derivative(Cm_biomass, t)
        Rm_CO2 = derivative(Cm_CO2, t)
        Rm_acetic_acid = derivative(Cm_acetic_acid, t)
        
        rates_and_yields['R glucose [g/L/h]'].extend(Rm_glucose)
        rates_and_yields['R O2 [g/L/h]'].extend(Rm_O2)
        rates_and_yields['R base [g/L/h]'].extend(Rm_base)
        rates_and_yields['R biomass [g/L/h]'].extend(Rm_biomass)
        rates_and_yields['R CO2 [g/L/h]'].extend(Rm_CO2)
        rates_and_yields['R acetic acid [g/L/h]'].extend(Rm_acetic_acid)
                    
        if reactor_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            Rm_tryptophan = derivative(Cm_tryptophan, t)
            rates_and_yields['R tryptophan [g/L/h]'].extend(Rm_tryptophan)
        else:
            rates_and_yields['R tryptophan [g/L/h]'].extend([None] * len(t))

        # [mol * L^-1 * h^-1]
        R_glucose = derivative(C_glucose, t)
        R_O2 = derivative(C_O2, t)
        R_base = derivative(C_base, t)
        R_biomass = derivative(C_biomass, t)
        R_CO2 = derivative(C_CO2, t)
        R_acetic_acid = derivative(C_acetic_acid, t)
        
        rates_and_yields['R glucose [mol/L/h]'].extend(R_glucose)
        rates_and_yields['R O2 [mol/L/h]'].extend(R_O2)
        rates_and_yields['R base [mol/L/h]'].extend(R_base)
        rates_and_yields['R biomass [mol/L/h]'].extend(R_biomass)
        rates_and_yields['R CO2 [mol/L/h]'].extend(R_CO2)
        rates_and_yields['R acetic acid [mol/L/h]'].extend(R_acetic_acid)

        if reactor_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            R_tryptophan = derivative(C_tryptophan, t)
            rates_and_yields['R tryptophan [mol/L/h]'].extend(R_tryptophan)
        else:
            rates_and_yields['R tryptophan [mol/L/h]'].extend([None] * len(t))
    
        ### Biomass specific rates (q) 
        # [g * g biomass^-1 * h^-1]
        qm_glucose = Rm_glucose / Cm_biomass
        qm_O2 = Rm_O2 / Cm_biomass
        qm_base = Rm_base / Cm_biomass
        µ = Rm_biomass / Cm_biomass
        qm_CO2 = Rm_CO2 / Cm_biomass
        qm_acetic_acid = Rm_acetic_acid / Cm_biomass

        rates_and_yields['q glucose [g/g/h]'].extend(qm_glucose)
        rates_and_yields['q O2 [g/g/h]'].extend(qm_O2)
        rates_and_yields['q base [g/g/h]'].extend(qm_base)
        
        rates_and_yields['q biomass (µ) [1/h]'].extend(µ)

        rates_and_yields['q CO2 [g/g/h]'].extend(qm_CO2)
        rates_and_yields['q acetic acid [g/g/h]'].extend(qm_acetic_acid)

        if reactor_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            qm_tryptophan = Rm_tryptophan / Cm_biomass
            rates_and_yields['q tryptophan [g/g/h]'].extend(qm_tryptophan)
        else:
            rates_and_yields['q tryptophan [g/g/h]'].extend([None] * len(t))
                
        # [mol * mol biomass^-1 * h^-1]
        q_glucose = R_glucose / C_biomass
        q_O2 = R_O2 / C_biomass
        q_base = R_base / C_biomass
        q_CO2 = R_CO2 / C_biomass
        q_acetic_acid = R_acetic_acid / C_biomass
    
        rates_and_yields['q glucose [mol/mol/h]'].extend(q_glucose)
        rates_and_yields['q O2 [mol/mol/h]'].extend(q_O2)
        rates_and_yields['q base [mol/mol/h]'].extend(q_base)
        rates_and_yields['q CO2 [mol/mol/h]'].extend(q_CO2)
        rates_and_yields['q acetic acid [mol/mol/h]'].extend(q_acetic_acid)

        if reactor_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            q_tryptophan = R_tryptophan / C_biomass
            rates_and_yields['q tryptophan [mol/mol/h]'].extend(q_tryptophan)
        else:
            rates_and_yields['q tryptophan [mol/mol/h]'].extend([None] * len(t))
        
        ## Yields
        Ym_biomass =  AMBR_data_df['Biomass [g]'] / AMBR_data_df['Consumed D-glucose [g]']                # [g/g of glucose]
        Y_biomass =  1e-3*AMBR_data_df['Biomass [mmol]'] / 1e-3*AMBR_data_df['Consumed D-glucose [mmol]'] # [mol/mol of glucose]
        
        rates_and_yields['Y biomass [g/g]'].extend(Ym_biomass)
        rates_and_yields['Y biomass [mol/mol]'].extend(Y_biomass)

        Ym_acetic_acid =  AMBR_data_df['acetic acid [g]'] / AMBR_data_df['Consumed D-glucose [g]']                # [g/g of glucose]
        Y_acetic_acid =  1e-3*AMBR_data_df['acetic acid [mmol]'] / 1e-3*AMBR_data_df['Consumed D-glucose [mmol]'] # [mol/mol of glucose]
            
        rates_and_yields['Y acetic acid [g/g]'].extend(Ym_acetic_acid)
        rates_and_yields['Y acetic acid [mol/mol]'].extend(Y_acetic_acid)

        Ym_CO2 =  AMBR_data_df['Formed CO2 [g]'] / AMBR_data_df['Consumed D-glucose [g]']                # [g/g of glucose]
        Y_CO2 =  1e-3*AMBR_data_df['Formed CO2 [mmol]'] / 1e-3*AMBR_data_df['Consumed D-glucose [mmol]'] # [mol/mol of glucose]
            
        rates_and_yields['Y CO2 [g/g]'].extend(Ym_CO2)
        rates_and_yields['Y CO2 [mol/mol]'].extend(Y_CO2)

        if reactor_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            Ym_tryptophan =  AMBR_data_df['tryptophan [g]'] / AMBR_data_df['Consumed D-glucose [g]']                # [g/g of glucose]
            Y_tryptophan =  1e-3*AMBR_data_df['tryptophan [mmol]'] / 1e-3*AMBR_data_df['Consumed D-glucose [mmol]'] # [mol/mol of glucose]
        
            rates_and_yields['Y tryptophan [g/g]'].extend(Ym_tryptophan)
            rates_and_yields['Y tryptophan [mol/mol]'].extend(Y_tryptophan)
        
        else:
            rates_and_yields['Y tryptophan [g/g]'].extend([None] * len(t))
            rates_and_yields['Y tryptophan [mol/mol]'].extend([None] * len(t))
        
        rates_and_yields_df = pd.DataFrame(rates_and_yields)
        rates_and_yields_dfs[reactor_name] = rates_and_yields_df
        
        printText("Reactor: ", Fore.BLACK, f"{reactor_name}", Fore.BLUE)
        display(rates_and_yields_df)

def plotConsumptionAndProductionRatesIndividual(number_of_reactors=12, folder_name=FOLDER_NAME_4):
    global rates_and_yields_dfs, AMBR_data_dfs
    
    styles = {'glucose': {'marker': 's', 'color': 'blue'}, 'acetic acid': {'marker': 'D', 'color': 'orange'}, 'tryptophan': {'marker': 'P', 'color': 'magenta'}, 'base': {'marker': 'v', 'color': 'purple'}, 'O2': {'marker': 'p', 'color': 'red'}, 'CO2': {'marker': '^', 'color': 'brown'}, 'biomass': {'marker': 'h', 'color': 'green'}}
    rates_y_lim_right = {"glucose": 35, "O2": 6, "base": 1, "biomass": 7,  "CO2": 10, "acetic acid": 0.225,  "tryptophan": 0.130}
    
    for i in range(0, number_of_reactors, 3):
        reactor_names = list(rates_and_yields_dfs.keys())[i:i+3]
        start_feed_time_r_1 = findStartFeed(AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(AMBR_data_dfs[reactor_names[2]])
        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3
        
        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)
        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
                
        for compound_name in styles.keys():
            if not compound_name=="tryptophan" and compound_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:

                fig, ax_left = plt.subplots(figsize=(8*0.85, 5*0.85))
                ax_right = ax_left.twinx()
                
                ax_left_title = f'R {compound_name} [mol/L/h]'
                ax_right_title = f'R {compound_name} [g/L/h]'
                
                process_time_r_1 = rates_and_yields_dfs[reactor_names[0]]["Process Time [h]"]
                process_time_r_2 = rates_and_yields_dfs[reactor_names[1]]["Process Time [h]"]
                process_time_r_3 = rates_and_yields_dfs[reactor_names[2]]["Process Time [h]"]
                avg_process_time = np.mean([process_time_r_1, process_time_r_2, process_time_r_3], axis=0)

                values_r_1_ax_left = rates_and_yields_dfs[reactor_names[0]][ax_left_title]
                values_r_2_ax_left = rates_and_yields_dfs[reactor_names[1]][ax_left_title]
                values_r_3_ax_left = rates_and_yields_dfs[reactor_names[2]][ax_left_title]
                avg_values_ax_left = np.mean([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)
                std_values_ax_left = np.std([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)

                values_r_1_ax_right = rates_and_yields_dfs[reactor_names[0]][ax_right_title]
                values_r_2_ax_right = rates_and_yields_dfs[reactor_names[1]][ax_right_title]
                values_r_3_ax_right = rates_and_yields_dfs[reactor_names[2]][ax_right_title]
                avg_values_ax_right = np.mean([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)
                std_values_ax_right = np.std([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)

                ax_left.plot(avg_process_time-avg_start_feed_time, avg_values_ax_left, marker=styles[compound_name]["marker"], linestyle='-', color=styles[compound_name]["color"], alpha=0.55)
                ax_left.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_left, yerr=std_values_ax_left, color='black', capsize=7, linestyle="", alpha=0.35)
                
                ax_right.plot(avg_process_time-avg_start_feed_time, avg_values_ax_right, color='blue', alpha=0.0) # invisible
                ax_right.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_right, yerr=std_values_ax_right, color='blue', alpha=0.0, capsize=7, linestyle="") # invisible

                
                compound_name_str = "CO2" if compound_name == "CO2" else compound_name.capitalize()
                rate_type = "consumption rate" if compound_name in ["glucose", "O2", "base"] else "production rate"

                ax_left.set_title(f"\n{compound_name_str} {rate_type}\n[g · L⁻¹ · h⁻¹]    |    [mol · L⁻¹ · h⁻¹]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}\n")

                ax_left.set_xlabel('Process Time [h]')
                
                ax_left.axvline(x=0, color='black', linestyle='--', alpha=0.35)  
                        
                ax_left.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)
                ax_right.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)

                ax_left.set_ylabel("[mol·L⁻¹·h⁻¹]", color="black")
                ax_right.set_ylabel("[g·L⁻¹·h⁻¹]", color="black")
                
                ax_left.tick_params(axis='y', labelcolor='black')
                ax_right.tick_params(axis='y', labelcolor='black')

                ax_left.tick_params(axis='x', labelcolor='black', which='both', bottom=True, top=True)

                ax_left.text(0.19, 0.95, 'Batch', transform=ax_left.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                ax_left.text(0.635, 0.95, 'Fed-batch', transform=ax_left.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                
                min_y_lim_left, max_y_lim_left = ax_left.get_ylim()[0], ax_left.get_ylim()[1]
                min_y_lim_right, max_y_lim_right = ax_right.get_ylim()[0], ax_right.get_ylim()[1]

                multiplication_factor_upper = rates_y_lim_right[compound_name]/abs(max_y_lim_right-min_y_lim_right)

                upper_y_limit_left = max_y_lim_left*multiplication_factor_upper*1.1 
                upper_y_limit_right = max_y_lim_right*multiplication_factor_upper*1.1
            
                lower_y_limit_left = min_y_lim_left - (abs(max_y_lim_left-min_y_lim_left)*0.105)
                lower_y_limit_right = min_y_lim_right - (abs(max_y_lim_right-min_y_lim_right)*0.105)
                
                ax_left.set_ylim(lower_y_limit_left, upper_y_limit_left)
                ax_right.set_ylim(lower_y_limit_right, upper_y_limit_right)

                ax_left.set_xlim(ax_left.get_xlim()[0]*1.0175, ax_left.get_xlim()[1]*1.0175)
                ax_right.set_xlim(ax_right.get_xlim()[0]*1.0175, ax_right.get_xlim()[1]*1.0175 )            


                if IS_SAVE_FIGURES:
                    plt.savefig(f'figures/{folder_name}/rates_{compound_name}_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')
                plt.show()

def plotBiomassSpecificConsumptionAndProductionRatesIndividual(number_of_reactors=12, folder_name=FOLDER_NAME_5):
    global rates_and_yields_dfs, AMBR_data_dfs
    
    styles = {
        'glucose': {'marker': 's', 'color': 'blue'},
        'acetic acid': {'marker': 'D', 'color': 'orange'},
        'tryptophan': {'marker': 'P', 'color': 'magenta'},
        'base': {'marker': 'v', 'color': 'purple'},
        'O2': {'marker': 'p', 'color': 'red'},
        'CO2': {'marker': '^', 'color': 'brown'},
        'biomass (µ)': {'marker': 'h', 'color': 'green'}
    }

    q_rates_y_lim_right = {
        "glucose": 4.50,
        "O2": 1.00,
        "base": 0.25,
        "biomass (µ)": 1.50, 
        "CO2": 1.00,
        "acetic acid": 0.30, 
        "tryptophan": 0.030
    }
 
    for i in range(0, number_of_reactors, 3):
        reactor_names = list(rates_and_yields_dfs.keys())[i:i+3]
        start_feed_time_r_1 = findStartFeed(AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(AMBR_data_dfs[reactor_names[2]])
        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3
        
        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)
        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
                
        for compound_name in styles.keys():
            if not compound_name=="tryptophan" and compound_name not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:

                fig, ax_left = plt.subplots(figsize=(8*0.85, 5*0.85))
                ax_right = ax_left.twinx()
                
                if not compound_name == "biomass (µ)":
                    ax_left_title, ax_right_title = f'q {compound_name} [mol/mol/h]', f'q {compound_name} [g/g/h]'
                else:
                    ax_left_title, ax_right_title= f'q biomass (µ) [1/h]', f'q biomass (µ) [1/h]'
                
                process_time_r_1 = rates_and_yields_dfs[reactor_names[0]]["Process Time [h]"]
                process_time_r_2 = rates_and_yields_dfs[reactor_names[1]]["Process Time [h]"]
                process_time_r_3 = rates_and_yields_dfs[reactor_names[2]]["Process Time [h]"]
                avg_process_time = np.mean([process_time_r_1, process_time_r_2, process_time_r_3], axis=0)

                values_r_1_ax_left = rates_and_yields_dfs[reactor_names[0]][ax_left_title]
                values_r_2_ax_left = rates_and_yields_dfs[reactor_names[1]][ax_left_title]
                values_r_3_ax_left = rates_and_yields_dfs[reactor_names[2]][ax_left_title]
                avg_values_ax_left = np.mean([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)
                std_values_ax_left = np.std([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)

                values_r_1_ax_right = rates_and_yields_dfs[reactor_names[0]][ax_right_title]
                values_r_2_ax_right = rates_and_yields_dfs[reactor_names[1]][ax_right_title]
                values_r_3_ax_right = rates_and_yields_dfs[reactor_names[2]][ax_right_title]
                avg_values_ax_right = np.mean([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)
                std_values_ax_right = np.std([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)

                ax_left.plot(avg_process_time-avg_start_feed_time, avg_values_ax_left, marker=styles[compound_name]["marker"], linestyle='-', color=styles[compound_name]["color"], alpha=0.55)
                ax_left.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_left, yerr=std_values_ax_left, color='black', capsize=7, linestyle="", alpha=0.35)
                
                ax_right.plot(avg_process_time-avg_start_feed_time, avg_values_ax_right, color='blue', alpha=0.0) # invisible
                ax_right.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_right, yerr=std_values_ax_right, color='blue', alpha=0.0, capsize=7, linestyle="") # invisible
                
                compound_name_str = "CO2" if compound_name == "CO2" else compound_name.capitalize()
                q_rate_type = "biomass-specific consumption rate (q-rate)" if compound_name in ["glucose", "O2", "base"] else "biomass-specific production rate (q-rate)"
        
                if compound_name == "biomass (µ)":
                    ax_left.set_title(f"\nGrowth rate (µ) [h⁻¹]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}\n")
                else:
                    ax_left.set_title(f"\n{compound_name_str}\n{q_rate_type}\n[g · g⁻¹ dry biomass · h⁻¹]    |    [mol · mol⁻¹ dry biomass · h⁻¹]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}\n")

                ax_left.set_xlabel('Process Time [h]')
                
                ax_left.axvline(x=0, color='black', linestyle='--', alpha=0.35)  
                        
                ax_left.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)
                ax_right.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)

                
                ax_left.set_ylabel("[mol·mol⁻¹·h⁻¹]", color="black")
                ax_right.set_ylabel("[g·g⁻¹·h⁻¹]", color="black")
                
                ax_left.tick_params(axis='y', labelcolor='black')
                ax_right.tick_params(axis='y', labelcolor='black')

                ax_left.tick_params(axis='x', labelcolor='black', which='both', bottom=True, top=True)

                ax_left.text(0.19, 0.95, 'Batch', transform=ax_left.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                ax_left.text(0.635, 0.95, 'Fed-batch', transform=ax_left.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                
                min_y_lim_left, max_y_lim_left = ax_left.get_ylim()[0], ax_left.get_ylim()[1]
                min_y_lim_right, max_y_lim_right = ax_right.get_ylim()[0], ax_right.get_ylim()[1]

                multiplication_factor_upper = q_rates_y_lim_right[compound_name]/abs(max_y_lim_right-min_y_lim_right)

                upper_y_limit_left = max_y_lim_left*multiplication_factor_upper*1.1 
                upper_y_limit_right = max_y_lim_right*multiplication_factor_upper*1.1
            
                lower_y_limit_left = min_y_lim_left - (abs(max_y_lim_left-min_y_lim_left)*0.105)
                lower_y_limit_right = min_y_lim_right - (abs(max_y_lim_right-min_y_lim_right)*0.105)
                
                ax_left.set_ylim(lower_y_limit_left, upper_y_limit_left)
                ax_right.set_ylim(lower_y_limit_right, upper_y_limit_right)

                ax_left.set_xlim(ax_left.get_xlim()[0]*1.0175, ax_left.get_xlim()[1]*1.0175)
                ax_right.set_xlim(ax_right.get_xlim()[0]*1.0175, ax_right.get_xlim()[1]*1.0175 )            

                if IS_SAVE_FIGURES:
                    plt.savefig(f'figures/{folder_name}/q_rates_{compound_name}_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')
                plt.show()

def plotConsumptionAndProductionRatesSubplot(number_of_reactors=12, folder_name=FOLDER_NAME_4):
    global AMBR_data_dfs, rates_and_yields_dfs
    
    styles = {'glucose': {'marker': 's', 'color': 'blue'}, 'acetic acid': {'marker': 'D', 'color': 'orange'}, 'tryptophan': {'marker': 'P', 'color': 'magenta'}, 'base': {'marker': 'v', 'color': 'purple'}, 'O2': {'marker': 'p', 'color': 'red'}, 'CO2': {'marker': '^', 'color': 'brown'}, 'biomass': {'marker': 'h', 'color': 'green'}}
    rates_y_lim_right = {"glucose": 35, "O2": 6, "base": 1, "biomass": 7,  "CO2": 10, "acetic acid": 0.225,  "tryptophan": 0.130}
    
    for i in range(0, number_of_reactors, 3):
        reactor_names = list(rates_and_yields_dfs.keys())[i:i+3]
        
        start_feed_time_r_1 = findStartFeed(AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(AMBR_data_dfs[reactor_names[2]])

        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3

        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10*0.90, 14*0.90), sharex=True)
        else:
            fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10*0.90, 14*0.90*3/4), sharex=True)
        
        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
        fig.suptitle(f"\nConsumption and production rates\n[g · L⁻¹ · h⁻¹]    |    [mol · L⁻¹ · h⁻¹]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}\n")
        
        ax1_right, ax1_left = 'R glucose [g/L/h]', 'R glucose [mol/L/h]'
        ax2_right, ax2_left = 'R O2 [g/L/h]', 'R O2 [mol/L/h]'
        ax3_right, ax3_left = 'R base [g/L/h]', 'R base [mol/L/h]'
        ax4_right, ax4_left = 'R biomass [g/L/h]', 'R biomass [mol/L/h]'
        ax5_right, ax5_left = 'R CO2 [g/L/h]', 'R CO2 [mol/L/h]'
        ax6_right, ax6_left = 'R acetic acid [g/L/h]', 'R acetic acid [mol/L/h]'
        
        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            ax7_right, ax7_left = 'R tryptophan [g/L/h]', 'R tryptophan [mol/L/h]'
            ax8_right, ax8_left = None, None

            subplot_axes = [
                (ax1_right, ax1_left), (ax2_right, ax2_left), (ax3_right, ax3_left),
                (ax4_right, ax4_left), (ax5_right, ax5_left), (ax6_right, ax6_left),
                (ax7_right, ax7_left), (ax8_right, ax8_left)
            ]

        else:        
            subplot_axes = [
                (ax1_right, ax1_left), (ax2_right, ax2_left), (ax3_right, ax3_left),
                (ax4_right, ax4_left), (ax5_right, ax5_left), (ax6_right, ax6_left)
                ]
        
        for i, (ax_right, ax_left) in enumerate(subplot_axes):
            if ax_right and ax_left:
                
                ax_left_subplot = axes[i // 2, i % 2]
                ax_right_subplot = ax_left_subplot.twinx()

                compound_name = ax_right[2:].split(" [")[0]

                process_time_r_1 = rates_and_yields_dfs[reactor_names[0]]["Process Time [h]"]
                process_time_r_2 = rates_and_yields_dfs[reactor_names[1]]["Process Time [h]"]
                process_time_r_3 = rates_and_yields_dfs[reactor_names[2]]["Process Time [h]"]
                avg_process_time = np.mean([process_time_r_1, process_time_r_2, process_time_r_3], axis=0)

                values_r_1_ax_left = rates_and_yields_dfs[reactor_names[0]][ax_left]
                values_r_2_ax_left = rates_and_yields_dfs[reactor_names[1]][ax_left]
                values_r_3_ax_left = rates_and_yields_dfs[reactor_names[2]][ax_left]
                avg_values_ax_left = np.mean([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)
                std_values_ax_left = np.std([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)

                values_r_1_ax_right = rates_and_yields_dfs[reactor_names[0]][ax_right]
                values_r_2_ax_right = rates_and_yields_dfs[reactor_names[1]][ax_right]
                values_r_3_ax_right = rates_and_yields_dfs[reactor_names[2]][ax_right]
                avg_values_ax_right = np.mean([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)
                std_values_ax_right = np.std([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)

                ax_left_subplot.plot(avg_process_time-avg_start_feed_time, avg_values_ax_left, marker=styles[compound_name]["marker"], linestyle='-', color=styles[compound_name]["color"], alpha=0.55)
                ax_left_subplot.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_left, yerr=std_values_ax_left, color='black', capsize=7, linestyle="", alpha=0.35)
                
                ax_right_subplot.plot(avg_process_time-avg_start_feed_time, avg_values_ax_right, color='blue', alpha=0.0) # invisible
                ax_right_subplot.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_right, yerr=std_values_ax_right, color='blue', alpha=0.0, capsize=7, linestyle="") # invisible

                ax_left_subplot.set_title(compound_name)
                ax_left_subplot.set_xlabel('Process Time [h]')
                
                ax_left_subplot.axvline(x=0, color='black', linestyle='--', alpha=0.35)  
                        
                ax_left_subplot.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)
                ax_right_subplot.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)

                ax_left_subplot.set_ylabel("[mol·L⁻¹·h⁻¹]", color="black")
                ax_right_subplot.set_ylabel("[g·L⁻¹·h⁻¹]", color="black")
                
                ax_left_subplot.tick_params(axis='y', labelcolor='black')
                ax_right_subplot.tick_params(axis='y', labelcolor='black')

                ax_left_subplot.tick_params(axis='x', labelcolor='black', which='both', bottom=True, top=True)

                ax_left_subplot.text(0.19, 0.95, 'Batch', transform=ax_left_subplot.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                ax_left_subplot.text(0.635, 0.95, 'Fed-batch', transform=ax_left_subplot.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                
                min_y_lim_left, max_y_lim_left = ax_left_subplot.get_ylim()[0], ax_left_subplot.get_ylim()[1]
                min_y_lim_right, max_y_lim_right = ax_right_subplot.get_ylim()[0], ax_right_subplot.get_ylim()[1]

                multiplication_factor_upper = rates_y_lim_right[compound_name]/abs(max_y_lim_right-min_y_lim_right)

                upper_y_limit_left = max_y_lim_left*multiplication_factor_upper*1.1 
                upper_y_limit_right = max_y_lim_right*multiplication_factor_upper*1.1
            
                lower_y_limit_left = min_y_lim_left - (abs(max_y_lim_left-min_y_lim_left)*0.105)
                lower_y_limit_right = min_y_lim_right - (abs(max_y_lim_right-min_y_lim_right)*0.105)
                
                ax_left_subplot.set_ylim(lower_y_limit_left, upper_y_limit_left)
                ax_right_subplot.set_ylim(lower_y_limit_right, upper_y_limit_right)

                ax_left_subplot.set_xlim(ax_left_subplot.get_xlim()[0]*1.0175, ax_left_subplot.get_xlim()[1]*1.0175)
                ax_right_subplot.set_xlim(ax_right_subplot.get_xlim()[0]*1.0175, ax_right_subplot.get_xlim()[1]*1.0175 )            

        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            fig.delaxes(axes[3, 1])
        
        plt.tight_layout()
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/subplot_rates_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotBiomassSpecificConsumptionAndProductionRatesSubplot(number_of_reactors=12, folder_name=FOLDER_NAME_5):
    
    styles = {
        'glucose': {'marker': 's', 'color': 'blue'},
        'acetic acid': {'marker': 'D', 'color': 'orange'},
        'tryptophan': {'marker': 'P', 'color': 'magenta'},
        'base': {'marker': 'v', 'color': 'purple'},
        'O2': {'marker': 'p', 'color': 'red'},
        'CO2': {'marker': '^', 'color': 'brown'},
        'biomass (µ)': {'marker': 'h', 'color': 'green'}
    }

    q_rates_y_lim_right = {
        "glucose": 4.50,
        "O2": 1.00,
        "base": 0.25,
        "biomass (µ)": 1.50, 
        "CO2": 1.00,
        "acetic acid": 0.30, 
        "tryptophan": 0.030
    }

    for i in range(0, number_of_reactors, 3):
        reactor_names =list(rates_and_yields_dfs.keys())[i:i+3]

        start_feed_time_r_1 = findStartFeed(AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(AMBR_data_dfs[reactor_names[2]])

        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3

        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10*0.90, 14*0.90), sharex=True)
        else:
            fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10*0.90, 14*0.90*3/4), sharex=True)
        
        
        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
        fig.suptitle(f"\nBiomass-specific production and consumption rates (q-rates)\n[g · g⁻¹ dry biomass · h⁻¹]    |    [mol · mol⁻¹ dry biomass · h⁻¹]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}\n")
        
        ax1_right, ax1_left = 'q glucose [g/g/h]', 'q glucose [mol/mol/h]'
        ax2_right, ax2_left = 'q O2 [g/g/h]', 'q O2 [mol/mol/h]'
        ax3_right, ax3_left = 'q base [g/g/h]', 'q base [mol/mol/h]'
        ax4_right, ax4_left = 'q biomass (µ) [1/h]', 'q biomass (µ) [1/h]'
        ax5_right, ax5_left = 'q CO2 [g/g/h]', 'q CO2 [mol/mol/h]'
        ax6_right, ax6_left = 'q acetic acid [g/g/h]', 'q acetic acid [mol/mol/h]'
        
        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            ax7_right, ax7_left = 'q tryptophan [g/g/h]', 'q tryptophan [mol/mol/h]'
            ax8_right, ax8_left = None, None

            subplot_axes = [
                (ax1_right, ax1_left), (ax2_right, ax2_left), (ax3_right, ax3_left),
                (ax4_right, ax4_left), (ax5_right, ax5_left), (ax6_right, ax6_left),
                (ax7_right, ax7_left), (ax8_right, ax8_left)
            ]
        else:        
            subplot_axes = [
                (ax1_right, ax1_left), (ax2_right, ax2_left), (ax3_right, ax3_left),
                (ax4_right, ax4_left), (ax5_right, ax5_left), (ax6_right, ax6_left)
            ]
        
        for i, (ax_right, ax_left) in enumerate(subplot_axes):
            if ax_right and ax_left:
                
                ax_left_subplot = axes[i // 2, i % 2]
                ax_right_subplot = ax_left_subplot.twinx()

                compound_name = ax_right[2:].split(" [")[0]

                process_time_r_1 = rates_and_yields_dfs[reactor_names[0]]["Process Time [h]"]
                process_time_r_2 = rates_and_yields_dfs[reactor_names[1]]["Process Time [h]"]
                process_time_r_3 = rates_and_yields_dfs[reactor_names[2]]["Process Time [h]"]
                avg_process_time = np.mean([process_time_r_1, process_time_r_2, process_time_r_3], axis=0)

                values_r_1_ax_left = rates_and_yields_dfs[reactor_names[0]][ax_left]
                values_r_2_ax_left = rates_and_yields_dfs[reactor_names[1]][ax_left]
                values_r_3_ax_left = rates_and_yields_dfs[reactor_names[2]][ax_left]
                avg_values_ax_left = np.mean([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)
                std_values_ax_left = np.std([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)

                values_r_1_ax_right = rates_and_yields_dfs[reactor_names[0]][ax_right]
                values_r_2_ax_right = rates_and_yields_dfs[reactor_names[1]][ax_right]
                values_r_3_ax_right = rates_and_yields_dfs[reactor_names[2]][ax_right]
                avg_values_ax_right = np.mean([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)
                std_values_ax_right = np.std([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)

                ax_left_subplot.plot(avg_process_time-avg_start_feed_time, avg_values_ax_left, marker=styles[compound_name]["marker"], linestyle='-', color=styles[compound_name]["color"], alpha=0.55)
                ax_left_subplot.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_left, yerr=std_values_ax_left, color='black', capsize=7, linestyle="", alpha=0.35)
                
                ax_right_subplot.plot(avg_process_time-avg_start_feed_time, avg_values_ax_right, alpha=0.1, color='blue') # invisible
                ax_right_subplot.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_right, yerr=std_values_ax_right, alpha=0.1, color='blue', capsize=7, linestyle="") # invisible
                
                ax_left_subplot.set_title(compound_name)
                ax_left_subplot.set_xlabel('Process Time [h]')
                ax_left_subplot.set_ylabel("[mol·mol⁻¹·h⁻¹]", color="black")
                ax_left_subplot.tick_params(axis='y', labelcolor='black')
                
                ax_left_subplot.axvline(x=0, color='black', linestyle='--', alpha=0.35)            
                
                ax_left_subplot.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)
                ax_right_subplot.axhline(y=0, color='black', linestyle='-', alpha=0.35*0.5)

                ax_right_subplot.set_ylabel("[g·g⁻¹·h⁻¹]", color="black")
                ax_right_subplot.tick_params(axis='y', labelcolor='black')

                ax_left_subplot.tick_params(axis='x', labelcolor='black', which='both', bottom=False, top=True)

                ax_left_subplot.text(0.19, 0.95, 'Batch', transform=ax_left_subplot.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                ax_left_subplot.text(0.635, 0.95, 'Fed-batch', transform=ax_left_subplot.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                
                min_y_lim_left, max_y_lim_left = ax_left_subplot.get_ylim()[0], ax_left_subplot.get_ylim()[1]
                min_y_lim_right, max_y_lim_right = ax_right_subplot.get_ylim()[0], ax_right_subplot.get_ylim()[1]

                multiplication_factor_upper = q_rates_y_lim_right[compound_name]/abs(max_y_lim_right-min_y_lim_right)

                upper_y_limit_left = max_y_lim_left*multiplication_factor_upper*1.1 
                upper_y_limit_right = max_y_lim_right*multiplication_factor_upper*1.1
            
                lower_y_limit_left = min_y_lim_left - (abs(max_y_lim_left-min_y_lim_left)*0.105)
                lower_y_limit_right = min_y_lim_right - (abs(max_y_lim_right-min_y_lim_right)*0.105)
                
                ax_left_subplot.set_ylim(lower_y_limit_left, upper_y_limit_left)
                ax_right_subplot.set_ylim(lower_y_limit_right, upper_y_limit_right)

                ax_left_subplot.set_xlim(ax_left_subplot.get_xlim()[0]*1.0175, ax_left_subplot.get_xlim()[1]*1.0175)
                ax_right_subplot.set_xlim(ax_right_subplot.get_xlim()[0]*1.0175, ax_right_subplot.get_xlim()[1]*1.0175 )            


        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            fig.delaxes(axes[3, 1])
        
        plt.tight_layout()
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/subplot_q_rates_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotSubstrateAndProductYieldsSubplot(number_of_reactors=12, folder_name=FOLDER_NAME_6):
    
    styles = {
        'glucose': {'marker': 's', 'color': 'blue'},
        'acetic acid': {'marker': 'D', 'color': 'orange'},
        'tryptophan': {'marker': 'P', 'color': 'magenta'},
        'base': {'marker': 'v', 'color': 'purple'},
        'O2': {'marker': 'p', 'color': 'red'},
        'CO2': {'marker': '^', 'color': 'brown'},
        'biomass': {'marker': 'h', 'color': 'green'}
    }

    for i in range(0, number_of_reactors, 3):
        reactor_names = list(rates_and_yields_dfs.keys())[i:i+3]

        start_feed_time_r_1 = findStartFeed(AMBR_data_dfs[reactor_names[0]])
        start_feed_time_r_2 = findStartFeed(AMBR_data_dfs[reactor_names[1]])
        start_feed_time_r_3 = findStartFeed(AMBR_data_dfs[reactor_names[2]])

        avg_start_feed_time = (start_feed_time_r_1 + start_feed_time_r_2 + start_feed_time_r_3) / 3

        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10*0.70*1.2, 14*0.90*2/4*1.2), sharex=True)
        
        bacth_medium = media_per_reactor[reactor_names[0]][0]
        feed_medium = media_per_reactor[reactor_names[0]][1]
        iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

        reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
        fig.suptitle(f"\nYields\n[g · g⁻¹ glucose · h⁻¹]    |    [mol · mol⁻¹ glucose · h⁻¹]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}\n")

        ax1_right, ax1_left = 'Y biomass [g/g]', 'Y biomass [mol/mol]'
        ax2_right, ax2_left = 'Y acetic acid [g/g]', 'Y acetic acid [mol/mol]'
        ax3_right, ax3_left = 'Y CO2 [g/g]', 'Y CO2 [mol/mol]'
        
        if reactor_names[0] not in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            ax4_right, ax4_left = 'Y tryptophan [g/g]', 'Y tryptophan [mol/mol]'
            subplot_axes = [(ax1_right, ax1_left), (ax2_right, ax2_left), (ax3_right, ax3_left), (ax4_right, ax4_left)]
        else:        
            subplot_axes = [(ax1_right, ax1_left), (ax2_right, ax2_left), (ax3_right, ax3_left)]
        
        for i, (ax_right, ax_left) in enumerate(subplot_axes):
            if ax_right and ax_left:
                
                ax_left_subplot = axes[i // 2, i % 2]
                ax_right_subplot = ax_left_subplot.twinx()

                compound_name = ax_right[2:].split(" [")[0]

                process_time_r_1 = rates_and_yields_dfs[reactor_names[0]]["Process Time [h]"]
                process_time_r_2 = rates_and_yields_dfs[reactor_names[1]]["Process Time [h]"]
                process_time_r_3 = rates_and_yields_dfs[reactor_names[2]]["Process Time [h]"]
                avg_process_time = np.mean([process_time_r_1, process_time_r_2, process_time_r_3], axis=0)

                values_r_1_ax_left = rates_and_yields_dfs[reactor_names[0]][ax_left]
                values_r_2_ax_left = rates_and_yields_dfs[reactor_names[1]][ax_left]
                values_r_3_ax_left = rates_and_yields_dfs[reactor_names[2]][ax_left]
                avg_values_ax_left = np.mean([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)
                std_values_ax_left = np.std([values_r_1_ax_left, values_r_2_ax_left, values_r_3_ax_left], axis=0)

                values_r_1_ax_right = rates_and_yields_dfs[reactor_names[0]][ax_right]
                values_r_2_ax_right = rates_and_yields_dfs[reactor_names[1]][ax_right]
                values_r_3_ax_right = rates_and_yields_dfs[reactor_names[2]][ax_right]
                avg_values_ax_right = np.mean([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)
                std_values_ax_right = np.std([values_r_1_ax_right, values_r_2_ax_right, values_r_3_ax_right], axis=0)

                ax_left_subplot.plot(avg_process_time-avg_start_feed_time, avg_values_ax_left, marker=styles[compound_name]["marker"], linestyle='--', color="grey", alpha=0.35)
                ax_left_subplot.plot(avg_process_time-avg_start_feed_time, avg_values_ax_left, marker=styles[compound_name]["marker"], linestyle='--', color=styles[compound_name]["color"], alpha=0.35*0.5)
                ax_left_subplot.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_left, yerr=std_values_ax_left, color='black', capsize=7, linestyle="", alpha=0.30)
                
                ax_right_subplot.plot(avg_process_time-avg_start_feed_time, avg_values_ax_right, marker=styles[compound_name]["marker"], linestyle='-', color=styles[compound_name]["color"], alpha=0.55)
                ax_right_subplot.errorbar(avg_process_time-avg_start_feed_time, avg_values_ax_right, yerr=std_values_ax_right, color='black', capsize=7, linestyle="", alpha=0.30)
                
                ax_left_subplot.set_title(compound_name)
                ax_left_subplot.set_xlabel('Process Time [h]')
                ax_left_subplot.set_ylabel("mol·mol⁻¹·h⁻¹ dashed line", color="black")
                ax_left_subplot.tick_params(axis='y', labelcolor='black')
                
                ax_left_subplot.axvline(x=0, color='black', linestyle='--', alpha=0.35)            
                ax_left_subplot.axhline(y=0, color='black', linestyle='-', alpha=0.35)

                ax_right_subplot.set_ylabel("g·g⁻¹·h⁻¹", color="black")
                ax_right_subplot.tick_params(axis='y', labelcolor='black')

                ax_left_subplot.tick_params(axis='x', labelcolor='black', which='both', bottom=True, top=True)

                ax_left_subplot.text(0.19, 0.95, 'Batch', transform=ax_left_subplot.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                ax_left_subplot.text(0.635, 0.95, 'Fed-batch', transform=ax_left_subplot.transAxes, fontsize=9, verticalalignment='top', color='black', alpha=0.75)
                
                min_y_lim_left, max_y_lim_left = ax_left_subplot.get_ylim()[0], ax_left_subplot.get_ylim()[1]
                min_y_lim_right, max_y_lim_right = ax_right_subplot.get_ylim()[0], ax_right_subplot.get_ylim()[1]

                yields_y_lim_right = {
                    "biomass": 5.0,
                    "acetic acid": 0.475, 
                    "CO2": 0.8, 
                    "tryptophan": 0.0275
                }
                multiplication_factor_upper = yields_y_lim_right[compound_name]/abs(max_y_lim_right-min_y_lim_right)

                upper_y_limit_left = max_y_lim_left*multiplication_factor_upper*1.1 
                upper_y_limit_right = max_y_lim_right*multiplication_factor_upper*1.1
            
                lower_y_limit_left = 0 - (upper_y_limit_left*0.05)
                lower_y_limit_right = 0 - (upper_y_limit_right*0.05)
                
                ax_left_subplot.set_ylim(lower_y_limit_left, upper_y_limit_left)
                ax_right_subplot.set_ylim(lower_y_limit_right, upper_y_limit_right)

                ax_left_subplot.set_xlim(ax_left_subplot.get_xlim()[0]*1.0175, ax_left_subplot.get_xlim()[1]*1.0175)
                ax_right_subplot.set_xlim(ax_right_subplot.get_xlim()[0]*1.0175, ax_right_subplot.get_xlim()[1]*1.0175 )            

        if reactor_names[0] in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"]:
            fig.delaxes(axes[1, 1])
        
        plt.tight_layout()
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/subplot_yields_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')

        plt.show()


In [8]:
### Helping functions for getElementsQuantity() 

def getElementProportionPerCompound(atom, compound):
    """
    Returns the elemental proportion [g ELEMENT/ g COMPOUND] of the compound solution or gases present in the bioreactor, and for a specific atom.
    The data is described in "elemental_composition_compound.xlsx"
    The compounds are: glucose, citric acid, acetic acid, tryptophan, oxygen, carbon-dioxide, NaOH and biomass 
    
    """
    global elemental_composition_compounds_df
    
    index_of_atom = elemental_composition_compounds_df[elemental_composition_compounds_df["Metal ion"] == atom].index[0]
    return elemental_composition_compounds_df.loc[index_of_atom, compound]

def getBatchMediaPerReactor(reactor):
    """
    Returns the name of the batch medium used in the reactor based on the data from "media_per_reactor"
    
    """
    for media in media_per_reactor[reactor]:
        if "DDB_BM_" in media:
            return media

def getFeedMediaPerReactor(reactor):
    """
    Returns the name of the feed medium used in the reactor based on the data from "media_per_reactor"
    
    """
    for media in media_per_reactor[reactor]:
        if "DDB_FM_" in media:
            return media 

def getElementConcentrationPerMedium(atom, medium):
    """
    Returns the elemental composition of the medium [g ELEMENT/ L MEDIA] for a specific atom based on the data from "elemental_composition_media.xlsx"

    """
    global elemental_composition_media_df
    index_of_atom = elemental_composition_media_df[elemental_composition_media_df["Metal ion"] == atom].index[0]
    return elemental_composition_media_df.loc[index_of_atom, medium]

def getMeasuredAtomList(df):
    """
    Returns a list of the measured elements from TM analysis of the supernatent or biomass described in "TM_biomass.xlsx" and "TM_supernatent.xlsx"
    
    """
    return [column.split(" ")[0] for column in df.columns[1:]]

def getTMValuesPerAtom(TM_data_df, reactor_name, atom):
    """
    Retruns the concentration values of a specific element obtained from the TM analysis of the supernatent or the biomass. 
    
    It returns also the unit used to describe the measured concentration given the units were either in "mg/L", "mg/g", "µg/L" or µg/g. 

    """

    TM_data = TM_data_df[TM_data_df['Sample name'].str.startswith(reactor_name)]

    for column in TM_data.columns:
        if atom in column:
            concentration_unit = column.split("[")[1].split("]")[0]
            concentrations = list(TM_data[column])
            return concentrations, concentration_unit

def geConvertionFactorToGramPerLiter(concentration_unit):
    """
    Retruns a conversion factor used to convert all units into either "g/L" or "g/g" 

    """
    if "µg" in concentration_unit:
        return 1000000.0
    elif "mg" in concentration_unit:
        return 1000.0
    else:
        print(f"Concentration unit unrecognized")
        return None

def getBiomassODValues(reactor_name):
    """
    Retruns the OD values for a specic reactor based on the data found in "manual_OD_measurements.xlsx"
    
    """
    global manual_OD_measurements_df
    
    biomass_data = manual_OD_measurements_df[manual_OD_measurements_df['sample name'].str.startswith(reactor_name)]
    return list(biomass_data["OD"])
    
def getCellDryWeighthPerLiter(OD_value):
    """ 
    Converts OD values into gram of biomass dry weight per litter (gDW/L)
    
    The value from Bionumbers for BL21 and BL21(DE3) E. coli strains is 1.0 OD600 = 0.39 gDW/L 

    Source: https://bionumbers.hms.harvard.edu/bionumber.aspx?s=n&v=3&id=109836

    The value from the used E. coli strains (HMP3071-004 and DDB35-002) in this experimenet is 1.0 OD600 = 0.3411 gDW/L
    
    """
    return 0.3411*OD_value

def getBiomassTotalCellVolume(OD_biomass):
    """ 
    Source: https://doi.org/10.1016/B978-0-08-099387-4.00005-3

    From the paper:

        1 mL sample of an E. coli culture with an OD600 = 1 has a total cell volume of 2.4-4.9 µL depending on cultivation conditions


    The max value is chosen = 4.9 µL

    """
    return 4.9*OD_biomass*1e-6  #[L]

def getDryBiomassWeight(reactor_name):
    """
    Retruns the dry weight values of the final sample given per a specific element described in "dry_weight_final_sample.xlsx" 
    
    """

    global dry_weight_final_sample_df
    dry_biomass_weight = dry_weight_final_sample_df[dry_weight_final_sample_df['Sample name'].str.startswith(reactor_name)]
    return list(dry_biomass_weight["Dry biomass [g]"])[0]

def createOrganizedDataPerReactor(number_of_reactors=12):
    """
    Returns an orginzied dataframe object containing only a sub-set the data generated by Ambr
    The data is specific to the reactor in question and the set of selected colomns deemed necessary for data analysis  

    """
    global AMBR_data_dfs, organized_AMBR_data_dfs
    
    for reactor_name, reactor_df in list(AMBR_data_dfs.items())[:number_of_reactors]:
        
        columns_list_based_on_sampling = [
            'Process Time [h]',
            'Sample volume [mL]', 
            'D-glucose [g/L]', 
            'citric acid [g/L]', 
            'acetic acid [g/L]',
            'tryptophan [g/L]'
            ]

        columns_list_continoues = [
            'Liquid volume [L]',
            'Feed 1 volume [L]',
            'Consumed O2 [mol]',
            'Formed CO2 [mol]',
            'Base volume [L]',
            "CTR [mol/h]",
            "OTR [mol/h]"
            ]

        organized_data = {
            'Process Time [h]': [],
            'Liquid volume [L]': [],
            
            'Biomass [unitless]': [],
            'Biomass [g/L]': [],
            'Biomass [g]': [],
            'Biomass [mol/L]': [], 
            'Biomass [mmol]': [],
            
            'D-glucose [g/L]': [],
            'Consumed D-glucose [g]': [],
            'D-glucose [mol/L]': [], 
            'Consumed D-glucose [mmol]': [],
            
            'citric acid [g/L]': [],
            'citric acid [g]': [],
            'citric acid [mol/L]': [], 
            'citric acid [mmol]': [],
            
            'acetic acid [g/L]': [],
            'acetic acid [g]': [],
            'acetic acid [mol/L]': [], 
            'acetic acid [mmol]': [],
            
            'tryptophan [g/L]': [],
            'tryptophan [g]': [],
            'tryptophan [mol/L]': [], 
            'tryptophan [mmol]': [],
            
            'Base volume [L]': [],
            'Base [g/L]': [],
            'Base [g]': [],
            'Base [mol/L]': [], 
            'Base [mmol]': [],
            
            'Feed 1 volume [L]': [],

            'Consumed O2 [g/L]': [],
            'Consumed O2 [g]': [],
            'Consumed O2 [mol/L]': [], 
            'Consumed O2 [mol]': [],
            'Consumed O2 [mmol]': [],

            'Formed CO2 [g/L]': [],
            'Formed CO2 [g]': [],
            'Formed CO2 [mol/L]': [], 
            'Formed CO2 [mol]': [],
            'Formed CO2 [mmol]': [],

            'CTR [mol/h]': [],
            'OTR [mol/h]': [],
            'Sample volume [mL]': [],
        }
        
        sampling_indices = []
        
        initial_time_index = reactor_df["Process Time [h]"].index[0]
        final_time_index = reactor_df["Process Time [h]"].index[-1]

        sampling_indices.append(initial_time_index)
        sampling_indices.extend(list(reactor_df[reactor_df["Sample volume [mL]"].notna()].index))
        
        for column in columns_list_continoues:
            for sampling_index in sampling_indices:
                if sampling_index in [sampling_indices[0], sampling_indices[-1]]:
                    value = reactor_df.at[sampling_index, column]
                    organized_data[column].append(value if pd.notna(value) else 0.0)
                else:
                    average_value = (reactor_df.at[sampling_index-1, column] + reactor_df.at[sampling_index+1, column]) / 2
                    organized_data[column].append(average_value)
                        
        for column in columns_list_based_on_sampling:
            for sampling_index in sampling_indices:
                if reactor_name in ["139_R06", "139_R07", "139_R08", "139_R09", "139_R10", "139_R11"] and column == 'tryptophan [g/L]':
                    organized_data[column].append(0.0)
                else:
                    organized_data[column].append(reactor_df.loc[sampling_index, column])
        
        times = organized_data["Process Time [h]"]
        volumes = organized_data["Liquid volume [L]"]        
        biomass_OD_values = getBiomassODValues(reactor_name)            # [unitless]
        
        for index, time in enumerate(times):
            
            if index == 0 or index == len(times)-1:       
                organized_data['Biomass [unitless]'].append(None)
                organized_data['Biomass [g/L]'].append(None)
                organized_data['Biomass [g]'].append(None)

                organized_data['Consumed D-glucose [g]'].append(None)
                organized_data['citric acid [g]'].append(None)
                organized_data['acetic acid [g]'].append(None)
                organized_data['tryptophan [g]'].append(None)
                
                organized_data['Biomass [mmol]'].append(None)
                organized_data['Consumed D-glucose [mmol]'].append(None)
                organized_data['citric acid [mmol]'].append(None)
                organized_data['acetic acid [mmol]'].append(None)
                organized_data['tryptophan [mmol]'].append(None)

                organized_data['Biomass [mol/L]'].append(None)
                organized_data['D-glucose [mol/L]'].append(None)
                organized_data['citric acid [mol/L]'].append(None)
                organized_data['acetic acid [mol/L]'].append(None)
                organized_data['tryptophan [mol/L]'].append(None)

            else:
                biomass_OD_value = biomass_OD_values[index-1]
                biomass_concentration = getCellDryWeighthPerLiter(biomass_OD_value)
                
                organized_data["Biomass [unitless]"].append(biomass_OD_value)            
                organized_data["Biomass [g/L]"].append(biomass_concentration)
                organized_data["Biomass [g]"].append(biomass_concentration*volumes[index])

                # C H(1.8) O(0.5) N (0.2)
                MW_biomass = 1.00*getMolarMass("C") + 1.80*getMolarMass("H") + 0.50*getMolarMass("O") + 0.20*getMolarMass("N")

                organized_data["Biomass [mol/L]"].append((biomass_concentration*volumes[index] / MW_biomass)/organized_data["Liquid volume [L]"][index])
                organized_data["Biomass [mmol]"].append(1e3*biomass_concentration*volumes[index] / MW_biomass)

                # In the batch medium: [C6H12O6*H2O] = 22 g/L 
                # In the feed medium: [C6H12O6*H2O] = 550 g/L

                glucose_conc_without_water_batch = getMolarMass("C6H12O6")/(getMolarMass("C6H12O6") + getMolarMass("H2O")) * 22 #g/L
                glucose_conc_without_water_fed_batch = getMolarMass("C6H12O6")/(getMolarMass("C6H12O6") + getMolarMass("H2O")) * 550 #g/L

                initial_volume_batch = organized_data["Liquid volume [L]"][0]

                if organized_data["Consumed D-glucose [g]"][index-1] == None:
                    consumed_glucose_batch = (initial_volume_batch*glucose_conc_without_water_batch) - (organized_data["D-glucose [g/L]"][index]*volumes[index])
                else:
                    consumed_glucose_batch = (initial_volume_batch*glucose_conc_without_water_batch) - (organized_data["D-glucose [g/L]"][index]*volumes[index]) + organized_data["Consumed D-glucose [g]"][index-1]
                
                consumed_glucose_fed_batch = glucose_conc_without_water_fed_batch*organized_data["Feed 1 volume [L]"][index]
                
                organized_data["Consumed D-glucose [g]"].append(consumed_glucose_batch+consumed_glucose_fed_batch)
                
                # In the feed: C6H8O7*H2O:	 1.86 g/L
                citric_acid_conc_without_water_in_feed = getMolarMass("C6H8O7")/(getMolarMass("C6H8O7") + getMolarMass("H2O")) * 1.86
                organized_data["citric acid [g]"].append((organized_data["citric acid [g/L]"][index] * volumes[index]) + (citric_acid_conc_without_water_in_feed*organized_data["Feed 1 volume [L]"][index]))
                
                organized_data["acetic acid [g]"].append(organized_data["acetic acid [g/L]"][index] * volumes[index])
                organized_data["tryptophan [g]"].append(organized_data["tryptophan [g/L]"][index] * volumes[index])

                organized_data["Consumed D-glucose [mmol]"].append(1e3*organized_data["Consumed D-glucose [g]"][index] / getMolarMass("C6H12O6"))
                organized_data["citric acid [mmol]"].append(1e3*organized_data["citric acid [g]"][index] / getMolarMass("C6H8O7"))
                organized_data["acetic acid [mmol]"].append(1e3*organized_data["acetic acid [g/L]"][index] * volumes[index] / getMolarMass("C2H4O2"))
                organized_data["tryptophan [mmol]"].append(1e3*organized_data["tryptophan [g/L]"][index] * volumes[index] / getMolarMass("C11H12N2O2"))

                organized_data["D-glucose [mol/L]"].append((organized_data["D-glucose [g/L]"][index] * volumes[index] / getMolarMass("C6H12O6"))/organized_data["Liquid volume [L]"][index])
                organized_data["citric acid [mol/L]"].append((organized_data["citric acid [g]"][index] / getMolarMass("C6H8O7"))/organized_data["Liquid volume [L]"][index])
                organized_data["acetic acid [mol/L]"].append((organized_data["acetic acid [g/L]"][index] * volumes[index] / getMolarMass("C2H4O2"))/organized_data["Liquid volume [L]"][index])
                organized_data["tryptophan [mol/L]"].append((organized_data["tryptophan [g/L]"][index] * volumes[index] / getMolarMass("C11H12N2O2"))/organized_data["Liquid volume [L]"][index])

            organized_data["Formed CO2 [g/L]"].append(organized_data["Formed CO2 [mol]"][index] * getMolarMass("CO2")/organized_data["Liquid volume [L]"][index])
            organized_data["Formed CO2 [g]"].append(organized_data["Formed CO2 [mol]"][index] * getMolarMass("CO2"))
            organized_data["Formed CO2 [mol/L]"].append(organized_data["Formed CO2 [mol]"][index]/organized_data["Liquid volume [L]"][index])
            organized_data["Formed CO2 [mmol]"].append(1e3*organized_data["Formed CO2 [mol]"][index])
            
            organized_data["Consumed O2 [g/L]"].append(organized_data["Consumed O2 [mol]"][index] * getMolarMass("O2")/organized_data["Liquid volume [L]"][index])
            organized_data["Consumed O2 [g]"].append(organized_data["Consumed O2 [mol]"][index] * getMolarMass("O2"))        
            organized_data["Consumed O2 [mol/L]"].append(organized_data["Consumed O2 [mol]"][index]/organized_data["Liquid volume [L]"][index])
            organized_data["Consumed O2 [mmol]"].append(1e3*organized_data["Consumed O2 [mol]"][index])
            
            NaOH_concentration = 2 # mol
            
            organized_data["Base [g/L]"].append(organized_data["Base volume [L]"][index] * NaOH_concentration * getMolarMass("NaOH")/organized_data["Liquid volume [L]"][index])
            organized_data["Base [g]"].append(organized_data["Base volume [L]"][index] * NaOH_concentration * getMolarMass("NaOH"))
            organized_data["Base [mol/L]"].append(organized_data["Base volume [L]"][index] * NaOH_concentration/organized_data["Liquid volume [L]"][index])
            organized_data["Base [mmol]"].append(1e3*organized_data["Base volume [L]"][index] * NaOH_concentration )
            
        organized_AMBR_data_dfs[reactor_name] = pd.DataFrame(organized_data)


#### functions for generating figures

7-Mass Balance Main Elements [mmol] vs [h]

8-Mass Balance Main Elements [%] vs [h]


In [9]:
def getElementsQuantity(compound_column, compound_name, organized_data_df, reactor_name=None, packed_data=None):
    """
    A very important function that calculates the elements quantity [ mmol ELEMENT ] transported or contained in a specific solution or gas:

    - Batch and feed mediums.
    - Compounds present in the reactor's broth such as glucose, citric acid, acetic acid, tryptophan and NaOH.
    - Biomass from OD measuremnets.
    - Gas streams such as O2 and CO2.
    - Trace metals present both in the supernatent and the last sample of dry biomass.
    
    The elements concerned are: C, H, O, N, S, P, Cl, Na, K, Mg, Ca, Fe, Mn, Zn, Cu, Co and Mo
    
    """
    global AMBR_data_dfs, TM_supernatent_df, TM_biomass_df
    global final_massic_concentration_per_element

    initial_time_point = AMBR_data_dfs[reactor_name]["Process Time [h]"].iloc[0]
    final_time_point = AMBR_data_dfs[reactor_name]["Process Time [h]"].iloc[-1]
    initial_volume = AMBR_data_dfs[reactor_name]["Liquid volume [L]"].iloc[0]
    final_volume = AMBR_data_dfs[reactor_name]["Liquid volume [L]"].iloc[-1]

    data_per_element = {}
            
    # Batch medium
    if compound_name == "Batch medium":

        # the first row of liquid volume = volume of the added batch medium
        used_batch_medium = getBatchMediaPerReactor(reactor_name)

        if initial_time_point not in data_per_element:
            data_per_element[initial_time_point] = []

        for atom in atoms:
            element_concentration = getElementConcentrationPerMedium(atom, used_batch_medium)  # [g ELEMENT/L]
            element_mass = element_concentration * initial_volume                              # [g ELEMENT]
            element_quantity = (element_mass/getMolarMass(atom))*1e3                           # [mmol ELEMENT]
            data_per_element[initial_time_point].append({atom: element_quantity})        

    # Feed medium
    elif compound_name == "Feed medium":
        
        for index, row in organized_data_df.iterrows():
            time_point = row["Process Time [h]"]
            volume = row["Feed 1 volume [L]"]

            used_feed_medium = getFeedMediaPerReactor(reactor_name)

            if time_point not in data_per_element:
                data_per_element[time_point] = []

            for atom in atoms:
                element_concentration = getElementConcentrationPerMedium(atom, used_feed_medium)  # [g ELEMENT/L]
                element_mass = element_concentration * volume                                     # [g ELEMENT]
                element_quantity = (element_mass/getMolarMass(atom))*1e3                          # [mmol ELEMENT]
                data_per_element[time_point].append({atom: element_quantity})        

    # Sampled volume 
    elif compound_name == "sampled volume":
        
        measured_atom_list = getMeasuredAtomList(TM_supernatent_df)
        
        for atom in atoms:
            ## The element was measured using HPLC
            if atom in measured_atom_list:
                for index, row in organized_data_df.iterrows():
                    
                    time_point = row["Process Time [h]"]
                    sample_volume = row["Sample volume [mL]"]*1e-3  # [L]
                    
                    if index == 0 or index == len(organized_data_df)-1:                         
                        if time_point not in data_per_element:
                            data_per_element[time_point] = []
                        data_per_element[time_point].append({atom: 0})

                    else:            
                        element_concentrations, concentration_unit = getTMValuesPerAtom(TM_supernatent_df, reactor_name, atom)  # [(µg or mg) METAL/L]
                        convertion_factor = geConvertionFactorToGramPerLiter(concentration_unit)
                        element_concentration = element_concentrations[index-1]/convertion_factor                               # [g METAL/L]

                        if time_point not in data_per_element:
                            data_per_element[time_point] = []
            
                        element_mass = element_concentration * sample_volume                                                    # [g METAL
                        element_quantity = (element_mass/getMolarMass(atom))*1e3                                                # [mmol METAL]

                        data_per_element[time_point].append({atom: element_quantity})
            else:
                ## The element was NOT measured using HPLC
                # ["C", "H", "O", "N", "S", "P", "Cl", "Na"]
                
                data_per_element_batch_medium_df, data_per_element_feed_medium_df, data_per_element_oxygen_df, data_per_element_base_df, data_per_element_co2_df = packed_data

                for index, row in organized_data_df.iterrows():
                    
                    time_point = row["Process Time [h]"]
                    reactor_volume = row["Liquid volume [L]"]       # [L]
                    sample_volume = row["Sample volume [mL]"]*1e-3  # [L]

                    if index == 0 or index == len(organized_data_df)-1:                         
                        if time_point not in data_per_element:
                            data_per_element[time_point] = []
                        data_per_element[time_point].append({atom: 0})

                    else:            
                        ratio_sample_over_reactor_volume = sample_volume/reactor_volume
                  
                        in_quantity = 0
                        in_quantity += data_per_element_batch_medium_df[atom][0]          # [mmol]
                        in_quantity += data_per_element_feed_medium_df[atom][index]       # [mmol]
                        in_quantity += data_per_element_oxygen_df[atom][index]            # [mmol]
                        in_quantity += data_per_element_base_df[atom][index]              # [mmol]

                        out_quantity = 0
                        out_quantity += data_per_element_co2_df[atom][index]              # [mmol]

                        accumulated_quantity = in_quantity-out_quantity                   # [mmol]
                        removed_quantity = accumulated_quantity*ratio_sample_over_reactor_volume # [mmol]
                        
                        if time_point not in data_per_element:
                            data_per_element[time_point] = []
            
                        data_per_element[time_point].append({atom: removed_quantity})

    # D-glucose, citric acid, acetic acid, tryptophan
    elif compound_name in ["Glucose", "Citric acid", "Acetic acid", "Tryptophan"]:
          
        for index, row in organized_data_df.iterrows():
            time_point = row["Process Time [h]"]
            volume = row["Liquid volume [L]"]
            concentration = row[compound_column]

            if time_point not in data_per_element:
                data_per_element[time_point] = []

            for atom in atoms:
                element_proportion = getElementProportionPerCompound(atom, compound_name)  # [g ELEMENT/g COMPOUND]
                
                mass_compound = volume * concentration                        # [g COMPOUND]
                element_mass = mass_compound * element_proportion             # [g ELEMENT]
                element_quantity = (element_mass/getMolarMass(atom))*1e3      # [mmol ELEMENT]

                data_per_element[time_point].append({atom: element_quantity})
    
    # Oxygen, carbon-dioxide
    elif compound_name in ["Oxygen", "Carbon-dioxide"]:

        for index, row in organized_data_df.iterrows():
            
            time_point = row["Process Time [h]"]
            
            quantity = row[compound_column]                             # [mol COMPOUND]

            if compound_name == "Oxygen":
                MW_compound = getMolarMass("O2")             
            elif compound_name == "Carbon-dioxide":
                MW_compound = getMolarMass("CO2")      

            if time_point not in data_per_element:
                data_per_element[time_point] = []

            for atom in atoms:
                element_proportion = getElementProportionPerCompound(atom, compound_name)  # [g ELEMENT/g COMPOUND]
                mass_compound = MW_compound * quantity                     # [g COMPOUND]
                element_mass = mass_compound * element_proportion          # [g ELEMENT]
                element_quantity = (element_mass/getMolarMass(atom))*1e3   # [mmol ELEMENT]

                data_per_element[time_point].append({atom: element_quantity})

    # NaOH
    elif compound_name == "NaOH":
        for index, row in organized_data_df.iterrows():
            time_point = row["Process Time [h]"]
            
            NaOH_added_volume = row[compound_column]                        # [L COMPOUND]
            NaOH_concentration = 2                                          # [mol COMPOUND /L]
            NaOH_quantity = NaOH_added_volume*NaOH_concentration            # [mol COMPOUND]

            if time_point not in data_per_element:
                data_per_element[time_point] = []

            for atom in atoms:
                element_proportion = getElementProportionPerCompound(atom, compound_name)      # [g ELEMENT/g COMPOUND]
                
                NaOH_mass = NaOH_quantity * getMolarMass("NaOH")             # [g COMPOUND]
                element_mass = NaOH_mass * element_proportion                # [g ELEMENT]
                element_quantity = (element_mass/getMolarMass(atom))*1e3     # [mmol ELEMENT]

                data_per_element[time_point].append({atom: element_quantity})
    
    # Biomass
    elif compound_name == "Biomass":

        biomass_OD_values = getBiomassODValues(reactor_name)            # [unitless]
        
        for index, row in organized_data_df.iterrows():
            time_point = row["Process Time [h]"]

            if index == 0 or index == len(organized_data_df)-1: 
                for atom in atoms:
                    if time_point not in data_per_element:
                        data_per_element[time_point] = []
                    data_per_element[time_point].append({atom: None})

            else:
                
                biomass_OD = biomass_OD_values[index-1]                              # [unitless]
                biomass_concentration = getCellDryWeighthPerLiter(biomass_OD)        # [gDW/L]

                reactor_volume = row["Liquid volume [L]"]                            # [L]
                biomass_mass = reactor_volume*biomass_concentration                          # [gDW] :  L x gDW x L^-1 = gDW

                if time_point not in data_per_element:
                    data_per_element[time_point] = []

                for atom in atoms:
                    element_proportion = getElementProportionPerCompound(atom, compound_name)     # [g ELEMENT/gDW]
                    element_mass = biomass_mass * element_proportion                 # [g ELEMENT]
                    element_quantity = (element_mass/getMolarMass(atom))*1e3         # [mmol ELEMENT]

                    data_per_element[time_point].append({atom: element_quantity})
        
    # TM supernatent
    elif compound_name == "TM supernatent":
         
        measured_atom_list = getMeasuredAtomList(TM_supernatent_df)
        biomass_OD_values = getBiomassODValues(reactor_name)            # [unitless]

        # accounting for the cell total volume (very small effect, max value at max OD is 0.5 mL)
        total_cell_volumes = []
        total_cell_volumes.append(0)
        for biomass_OD_value in biomass_OD_values:
            total_cell_volumes.append(getBiomassTotalCellVolume(biomass_OD_value))
        total_cell_volumes.append(total_cell_volumes[-1])

        for atom in atoms:

            if atom in measured_atom_list:
                for index, row in organized_data_df.iterrows():
                    
                    time_point = row["Process Time [h]"]
                    broth_volume = row["Liquid volume [L]"]

                    # Adding None to the first and last row to all atoms
                    if index == 0 or index == len(organized_data_df)-1: 
                        
                        if time_point not in data_per_element:
                            data_per_element[time_point] = []
                        data_per_element[time_point].append({atom: 0})

                    # Adding the calculated quantity per atom to each time point
                    else:            
                        element_concentrations, concentration_unit = getTMValuesPerAtom(TM_supernatent_df, reactor_name, atom)  # [(µg or mg) METAL/L]
                        convertion_factor = geConvertionFactorToGramPerLiter(concentration_unit)
                        element_concentration = element_concentrations[index-1]/convertion_factor                               # [g METAL/L]

                        if time_point not in data_per_element:
                            data_per_element[time_point] = []
            
                        element_mass = element_concentration * (broth_volume-total_cell_volumes[index-1])                                                           # [g METAL
                        element_quantity = (element_mass/getMolarMass(atom))*1e3                                                # [mmol METAL]

                        data_per_element[time_point].append({atom: element_quantity})
            else:
                for index, row in organized_data_df.iterrows():
                    time_point = row["Process Time [h]"]            
                    
                    if time_point not in data_per_element:
                        data_per_element[time_point] = []

                    data_per_element[time_point].append({atom: 0})

    # TM biomass
    elif compound_name == "TM biomass":

        if reactor_name not in final_massic_concentration_per_element:
            final_massic_concentration_per_element[reactor_name] = []

        measured_atom_list = getMeasuredAtomList(TM_biomass_df)
        for atom in atoms:
            if atom in measured_atom_list:

                dry_biomass_weight = getDryBiomassWeight(reactor_name)                                    # [gDW] -> taken from 40 mL BROTH
        
                reactor_volume = final_volume

                total_dry_biomass_weight = (reactor_volume/40)*dry_biomass_weight                         # [gDW] -> for all the reactor volume
                
                element_massic_concentrations, concentration_unit = getTMValuesPerAtom(TM_biomass_df, reactor_name, atom)  # [(µg or mg) ELEMENT / 1 gDW]
                concentration_divider = geConvertionFactorToGramPerLiter(concentration_unit)
                element_massic_concentration = element_massic_concentrations[0]/concentration_divider     # [g ELEMENT / 1 gDW]
                                
                final_massic_concentration_per_element[reactor_name].append({f"{atom} [µg/g]": element_massic_concentration*1e6})

                total_element_mass = total_dry_biomass_weight*element_massic_concentration
                total_element_quantity = (total_element_mass/getMolarMass(atom))*1e3                      # [mmol ELEMENT]

                if final_time_point not in data_per_element:
                    data_per_element[final_time_point] = []

                data_per_element[final_time_point].append({atom: total_element_quantity})
            
            else:

                if final_time_point not in data_per_element:
                    data_per_element[final_time_point] = []

                data_per_element[final_time_point].append({atom: 0})


    data_per_element_df = pd.DataFrame.from_dict({k: {list(dct.keys())[0]: list(dct.values())[0] for dct in data} for k, data in data_per_element.items()}, orient='index')

    data_per_element_df.index.name = 'Time Point'
    data_per_element_df.reset_index(inplace=True)
    return data_per_element_df

def setDataPerElementForAllElements(number_of_reactors=12):
    global mass_balance_data_per_elements, final_massic_concentration_per_element
    global organized_AMBR_data_dfs

    for reactor_name, organized_AMBR_data in list(organized_AMBR_data_dfs.items())[:number_of_reactors]:    
        data_per_element_batch_medium_df = getElementsQuantity(None, "Batch medium", organized_AMBR_data, reactor_name)
        data_per_element_feed_medium_df = getElementsQuantity("Feed 1 volume [L]", "Feed medium", organized_AMBR_data, reactor_name)
        data_per_element_biomass_df = getElementsQuantity(None, "Biomass", organized_AMBR_data, reactor_name)
        data_per_element_glucose_df = getElementsQuantity("D-glucose [g/L]", "Glucose", organized_AMBR_data, reactor_name)
        data_per_element_citric_acid_df = getElementsQuantity("citric acid [g/L]", "Citric acid", organized_AMBR_data, reactor_name)
        data_per_element_acetic_acid_df = getElementsQuantity("acetic acid [g/L]", "Acetic acid", organized_AMBR_data, reactor_name)
        data_per_element_tryptophan_df = getElementsQuantity("tryptophan [g/L]", "Tryptophan", organized_AMBR_data, reactor_name)
        data_per_element_oxygen_df = getElementsQuantity("Consumed O2 [mol]", "Oxygen", organized_AMBR_data, reactor_name)
        data_per_element_co2_df = getElementsQuantity("Formed CO2 [mol]", "Carbon-dioxide", organized_AMBR_data, reactor_name)
        data_per_element_base_df = getElementsQuantity("Base volume [L]", "NaOH", organized_AMBR_data, reactor_name)
        data_per_element_TM_supernatent_df = getElementsQuantity(None, "TM supernatent", organized_AMBR_data, reactor_name)
        data_per_element_TM_biomass_df = getElementsQuantity(None, "TM biomass", organized_AMBR_data, reactor_name)
        packed_data = [data_per_element_batch_medium_df, data_per_element_feed_medium_df, data_per_element_oxygen_df, data_per_element_base_df, data_per_element_co2_df]
        data_per_element_sampled_volume_df = getElementsQuantity(None, "sampled volume", organized_AMBR_data, reactor_name, packed_data)

        data_per_element = {
            "batch medium": data_per_element_batch_medium_df,
            "feed medium": data_per_element_feed_medium_df,
            "TM supernatent": data_per_element_TM_supernatent_df,
            "TM biomass": data_per_element_TM_biomass_df,
            "sampled volume": data_per_element_sampled_volume_df,
            "biomass": data_per_element_biomass_df,
            "glucose": data_per_element_glucose_df,
            "tryptophan": data_per_element_tryptophan_df,
            "citric acid": data_per_element_citric_acid_df,
            "acetic acid": data_per_element_acetic_acid_df,
            "O2": data_per_element_oxygen_df,
            "CO2": data_per_element_co2_df,
            "NaOH": data_per_element_base_df
        }
        mass_balance_data_per_elements[reactor_name] = data_per_element

def printElementQuantityPerSolutionTables(number_of_reactors=12, number_of_tables=13):
    """
    Used to display the generated tables of the quantity of elements per solution in [µmol]

    """
    for reactor_name, data_per_element in list(mass_balance_data_per_elements.items())[:number_of_reactors]:                
        printText(f"Reactor: {reactor_name}", Fore.RED)

        count = 0
        for compound_name, df in data_per_element.items():
            if count >= number_of_tables:
                break
            printText("Element quantity [mmol]: ", Fore.BLACK, f"{compound_name}", Fore.BLUE)
            
            if compound_name in ["TM supernantent", "TM biomass"]:
                pd.set_option('display.precision', 9)

            display(df)
            pd.set_option('display.precision', 6)

            if IS_SAVE_TABLES:
                with pd.ExcelWriter(f'tables/element_quantity_per_solution_{reactor_name}.xlsx') as writer:
                    for compound_name, df in data_per_element.items():
                        df.to_excel(writer, sheet_name=f"{compound_name}", index=False)

            count += 1

def calculateMassBalanceMainElements(number_of_reactors=12):
    global mass_balance_data_per_elements, main_elements
    global mass_balance_dfs

    for reactor_name in list(mass_balance_data_per_elements.keys())[:number_of_reactors]:

        data_per_element = {}

        time_points = mass_balance_data_per_elements[reactor_name]["feed medium"]["Time Point"]

        for i, time_point in enumerate(time_points):
            data_per_element[time_point] = []  

            for element in main_elements:
                
                in_quantity = 0
                in_quantity += mass_balance_data_per_elements[reactor_name]["batch medium"][element][0]      # [mmol]
                in_quantity += mass_balance_data_per_elements[reactor_name]["feed medium"][element][i]       # [mmol]
                in_quantity += mass_balance_data_per_elements[reactor_name]["O2"][element][i]                # [mmol]
                in_quantity += mass_balance_data_per_elements[reactor_name]["NaOH"][element][i]              # [mmol]

                out_quantity = 0
                out_quantity += mass_balance_data_per_elements[reactor_name]["CO2"][element][i]
                out_quantity += mass_balance_data_per_elements[reactor_name]["sampled volume"][element][i]

                accumulated_quantity = in_quantity-out_quantity

                explainable_accumulated_quantity = 0
                explainable_accumulated_quantity += mass_balance_data_per_elements[reactor_name]["biomass"][element][i]           # [mmol]
                explainable_accumulated_quantity += mass_balance_data_per_elements[reactor_name]["glucose"][element][i]           # [mmol]
                explainable_accumulated_quantity += mass_balance_data_per_elements[reactor_name]["tryptophan"][element][i]        # [mmol]
                explainable_accumulated_quantity += mass_balance_data_per_elements[reactor_name]["citric acid"][element][i]       # [mmol]
                explainable_accumulated_quantity += mass_balance_data_per_elements[reactor_name]["acetic acid"][element][i]       # [mmol]

                data_per_element[time_point].append({f"{element} (IN)": in_quantity})
                data_per_element[time_point].append({f"{element} (OUT)": -out_quantity})
                data_per_element[time_point].append({f"{element} (ACC)": accumulated_quantity})
                data_per_element[time_point].append({f"{element} (EXP)": explainable_accumulated_quantity})

        data_per_element_df = pd.DataFrame.from_dict({k: {list(dct.keys())[0]: list(dct.values())[0] for dct in data}for k, data in data_per_element.items()}, orient='index')
        data_per_element_df.index.name = 'Time Point'
        data_per_element_df.reset_index(inplace=True)
        mass_balance_dfs[reactor_name] = data_per_element_df

def plotMassBalanceMainElements(number_of_reactors=12, folder_name=FOLDER_NAME_7):
    global main_elements, mass_balance_dfs

    for element in main_elements:
        for reactor_name, df_mass_balance in list(mass_balance_dfs.items())[:number_of_reactors]:    
            
            bacth_medium, feed_medium = media_per_reactor[reactor_name][0], media_per_reactor[reactor_name][1]
            iron_zinc_condition, used_strain = getTitleInformation(reactor_name, bacth_medium, feed_medium)

            bar_width = 0.20
            bar_positions = np.arange(len(df_mass_balance['Time Point']))

            colors = {'IN': 'green', 'OUT': 'red', 'ACC': 'magenta','EXP': 'blue'}
            
            alphas = {'IN': 0.40, 'OUT': 0.40, 'ACC': 0.65,'EXP': 0.65}

            label_names = {'IN': 'IN', 'OUT': 'OUT', 'ACC': 'Accumulation','EXP': 'Measured accumulation'}
            fig, ax = plt.subplots()

            for value in colors:
                column_name = f'{element} ({value})'
                if column_name in df_mass_balance.columns:
                    values = df_mass_balance[column_name]
                    positions = bar_positions + (list(colors.keys()).index(value) - len(colors) / 2) * bar_width
                    ax.bar(positions[1:-1], values[1:-1], bar_width, label=label_names[value], color=colors[value], alpha=alphas[value])

            ax.set_xlabel('Time [h]')
            ax.set_ylabel('Quantity [mmol]')
            ax.set_title(f'{element}')
            
            vertical_line_pos = ((bar_positions[2] + bar_positions[3]) / 2) - (bar_positions[2] / 17)
            ax.axvline(x=vertical_line_pos, color='black', linestyle='-', alpha=0.95, linewidth=0.55)
            
            ax.set_xticks(bar_positions)
            ax.set_xticklabels([f'{round(label, 1):.0f}' for label in df_mass_balance['Time Point']])

            plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
            
            ax.spines['bottom'].set_visible(True)
            
            for pos in bar_positions:
                ax.axvline(x=pos, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

            ylim_values = {
                "C": (-225, 425), 
                "H": (-50, 1000), 
                "O": (-450, 950)
                }
            
            ax.set_ylim(ylim_values[element])    
            
            ax.text(0.065, 0.95, 'Batch phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.85)
            ax.text(0.55, 0.95, 'Feed phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.85)

            lines, labels = ax.get_legend_handles_labels()
            ax.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5, frameon=False)

            plt.title(f'Mass Balance: {element}\n  {reactor_name}  |  {used_strain}  |  {iron_zinc_condition}')
            
            if IS_SAVE_FIGURES:
                plt.savefig(f'figures/{folder_name}/MB_main_{element}_{reactor_name}.png', dpi=300, bbox_inches='tight')

            plt.show()

def plotMassBalanceMainElementsPercentageTriplicate(number_of_reactors = 12, folder_name=FOLDER_NAME_8):
    global main_elements, mass_balance_dfs

    for element in main_elements:
        for i in range(0, number_of_reactors, 3):
            reactor_names = list(mass_balance_dfs.keys())[i:i+3]
            
            bacth_medium, feed_medium = media_per_reactor[reactor_names[0]][0], media_per_reactor[reactor_names[0]][1]
            iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

            fig, ax = plt.subplots()

            bar_positions = np.arange(len(mass_balance_dfs[reactor_names[0]]["Time Point"]))

            time_points_reactor_1 = mass_balance_dfs[reactor_names[0]]["Time Point"]
            time_points_reactor_2 = mass_balance_dfs[reactor_names[1]]["Time Point"]
            time_points_reactor_3 = mass_balance_dfs[reactor_names[2]]["Time Point"]

            values_reactor_1 = mass_balance_dfs[reactor_names[0]][f'{element} (EXP)']/mass_balance_dfs[reactor_names[0]][f'{element} (ACC)']*100
            values_reactor_2 = mass_balance_dfs[reactor_names[1]][f'{element} (EXP)']/mass_balance_dfs[reactor_names[1]][f'{element} (ACC)']*100
            values_reactor_3 = mass_balance_dfs[reactor_names[2]][f'{element} (EXP)']/mass_balance_dfs[reactor_names[2]][f'{element} (ACC)']*100
            
            average_time_points = np.mean([time_points_reactor_1, time_points_reactor_2, time_points_reactor_3], axis=0)
            average_values = np.mean([values_reactor_1, values_reactor_2, values_reactor_3], axis=0)
            std_devs = np.std([values_reactor_1, values_reactor_2, values_reactor_3], axis=0)
                
            ax.bar(bar_positions[1:-1], average_values[1:-1], 0.20, color='black', alpha=0.70)
            ax.errorbar(bar_positions[1:-1], average_values[1:-1], yerr=std_devs[1:-1], color='red', capsize=7, linestyle="")

            vertical_line_pos = (bar_positions[2] + bar_positions[3]) / 2
            ax.axvline(x=vertical_line_pos, color='black', linestyle='-', alpha=0.95, linewidth=0.55)
            
            ax.fill_between(bar_positions, 0, 100, where=(bar_positions < 39), color='black', alpha=0.030)
            
            ax.axhline(y=100, color='black', linestyle='--', alpha=0.55)
            ax.set_ylim(0, 150)

            ax.set_xlabel('Time [h]')
            ax.set_ylabel('Percentage (%)')

            reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
                                                                    
            ax.set_title(f'Mass Balance (%): {element}\n{reactor_name_triplicate} | {used_strain} | {iron_zinc_condition}')

            ax.text(0.10, 0.95, 'Batch phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.80)
            ax.text(0.57, 0.95, 'Feed phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.80)

            ax.set_xticks(bar_positions)
            ax.set_xticklabels([f'{round(label, 1):.0f}' for label in average_time_points])
            
            ax.spines['bottom'].set_visible(True)

            for pos in bar_positions:
                ax.axvline(x=pos, color='gray', linestyle='--', linewidth=0.25)

            if IS_SAVE_FIGURES:
                plt.savefig(f'figures/{folder_name}/MB_main_triplicate_{element}_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')
            
            plt.show()


#### functions for generating figures:

9-Mass Balance TM Elements [mmol] vs [h]

10-Mass Balance TM Elements [%] vs [h]

In [10]:
def getLimitOfDetectionForTManalysis(element):
       
       global LODs
       max_reactor_volume = 0.123 # [L]

       concentration, concentration_unit= LODs[element].split(" ")                                      # [(µg or mg)/L ]
       concentration = float(concentration)/geConvertionFactorToGramPerLiter(concentration_unit)        # [g METAL/L]

       max_error_in_mass  = concentration * max_reactor_volume                    # [g METAL]
       max_error_in_quantity = (max_error_in_mass/getMolarMass(element))*1e6      # [µmol METAL]
       return max_error_in_quantity

def calculateMassBalanceTraceMetalElements(number_of_reactors=12):
    global mass_balance_data_per_elements, TM_elements
    global mass_balance_TM_elements_dfs

    for reactor_name in list(mass_balance_data_per_elements.keys())[:number_of_reactors]:
        data_per_element = {}
        time_points = mass_balance_data_per_elements[reactor_name]["feed medium"]["Time Point"]

        for i, time_point in enumerate(time_points):
            data_per_element[time_point] = []  

            for element in TM_elements:
                in_quantity = 0
                in_quantity += mass_balance_data_per_elements[reactor_name]["batch medium"][element][0] * 1000       # [µmol]
                in_quantity += mass_balance_data_per_elements[reactor_name]["feed medium"][element][i] * 1000        # [µmol]

                out_quantity = 0
                out_quantity += mass_balance_data_per_elements[reactor_name]["sampled volume"][element][i] * 1000    # [µmol]

                accumulated_total_quantity = in_quantity - out_quantity
                supernatent_quantity = mass_balance_data_per_elements[reactor_name]["TM supernatent"][element][i] * 1000  # [µmol]
                accumulated_in_biomass_quantity = accumulated_total_quantity-supernatent_quantity

                data_per_element[time_point].append({f"{element} (IN)": in_quantity})
                data_per_element[time_point].append({f"{element} (OUT)": -out_quantity})
                data_per_element[time_point].append({f"{element} (ACC)": accumulated_total_quantity})
                data_per_element[time_point].append({f"{element} (SUP)": supernatent_quantity})
                data_per_element[time_point].append({f"{element} (BIO)": accumulated_in_biomass_quantity})

        for element in TM_elements:
            ### Biomass final sample
            final_time_point = mass_balance_data_per_elements[reactor_name]["TM biomass"]["Time Point"][0] 
            final_TM_accumulated_in_biomass_quantity = mass_balance_data_per_elements[reactor_name]["TM biomass"][element][0]*1000        # µmol ELEMENT
            data_per_element[final_time_point].append({f"{element} (fBIO)": final_TM_accumulated_in_biomass_quantity})

        data_per_element_df = pd.DataFrame.from_dict({k: {list(dct.keys())[0]: list(dct.values())[0] for dct in data}for k, data in data_per_element.items()}, orient='index')

        data_per_element_df.index.name = 'Time Point'
        data_per_element_df.reset_index(inplace=True)
        mass_balance_TM_elements_dfs[reactor_name] = data_per_element_df

def plotMassBalanceTraceMetalElements(number_of_reactors=12, folder_name=FOLDER_NAME_9):
    global TM_elements, mass_balance_TM_elements_dfs

    for reactor_name, df_mass_balance in list(mass_balance_TM_elements_dfs.items())[:number_of_reactors]:
        for element in TM_elements:

            bacth_medium, feed_medium = media_per_reactor[reactor_name][0], media_per_reactor[reactor_name][1]
            iron_zinc_condition, used_strain = getTitleInformation(reactor_name, bacth_medium, feed_medium)

            bar_width = 0.20
            bar_positions = np.arange(len(df_mass_balance['Time Point']))
            colors = {'IN': 'green', 'OUT': 'red', 'ACC': 'magenta','SUP': 'blue', 'BIO': 'orange'}

            label_names = {'IN': 'IN', 'OUT': 'OUT', 'ACC': 'Accumulation','SUP': 'Accum. supernatent', 'BIO': 'Accum. biomass'}
            alphas = {'IN': 0.45, 'OUT': 0.45, 'ACC': 0.45,'SUP': 0.65, 'BIO': 0.65}

            fig, ax = plt.subplots()

            for value in colors:
                column_name = f'{element} ({value})'
                if column_name in df_mass_balance.columns:
                    values = df_mass_balance[column_name]
                    positions = bar_positions + (list(colors.keys()).index(value) - len(colors) / 2) * bar_width
                    ax.bar(positions[1:-1], values[1:-1], bar_width, label=label_names[value], color=colors[value], alpha=alphas[value])

            ax.set_xlabel('Time [h]')
            ax.set_ylabel('Quantity [µmol]')
            ax.set_title(f'{element}')
            
            vertical_line_pos = ((bar_positions[2] + bar_positions[3]) / 2) - ((bar_positions[3])/32)
            ax.axvline(x=vertical_line_pos, color='black', linestyle='-', alpha=0.95, linewidth=0.55)

            ax.set_xticks(bar_positions)
            ax.set_xticklabels([f'{round(label, 1):.0f}' for label in df_mass_balance['Time Point']])

            plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
            
            ax.spines['bottom'].set_visible(True)
            
            for pos in bar_positions:
                ax.axvline(x=pos, color='gray', linestyle='--', linewidth=0.5)

            ylim_values = {
                "Co": (-0.5, 1.5), 
                "Cu": (-0.5, 1.2), 
                "Fe": (-50, 120), 
                "K": (-2500, 14000), 
                "Mg": (-225, 1400), 
                "Mn": (-0.4, 1.0), 
                "Mo": (-0.4, 1.2), 
                "Zn": (-5.0, 12.5)
                }
            
            ax.set_ylim(ylim_values[element])    
            
            plt.axhline(y=getLimitOfDetectionForTManalysis(element), label=f"LOD {element}", color='red', linestyle='--', alpha=0.35)

            handles, labels = plt.gca().get_legend_handles_labels()
            lod_index = labels.index(f"LOD {element}")
            handles.append(handles.pop(lod_index))
            labels.append(labels.pop(lod_index))
            plt.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

            plt.title(f'Mass Balance: {element}\n  {reactor_name}  |  {used_strain}  |  {iron_zinc_condition}')
            
            ax.text(0.065, 0.95, 'Batch phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.85)
            ax.text(0.55, 0.95, 'Feed phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.85)
            
            if IS_SAVE_FIGURES:
                plt.savefig(f'figures/{folder_name}/MB_TM_{element}_{reactor_name}.png', dpi=300, bbox_inches='tight')

            plt.show()

def plotMassBalanceTMElementsPercentageTriplicate(number_of_reactors=12, folder_name=FOLDER_NAME_10):
    global TM_elements, mass_balance_TM_elements_dfs

    for element in TM_elements:
        for i in range(0, number_of_reactors, 3):
            reactor_names = list(mass_balance_TM_elements_dfs.keys())[i:i+3]
            
            bacth_medium, feed_medium = media_per_reactor[reactor_names[0]][0], media_per_reactor[reactor_names[0]][1]
            iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

            fig, ax = plt.subplots()

            bar_positions = np.arange(len(mass_balance_TM_elements_dfs[reactor_names[0]]["Time Point"]))

            time_points_reactor_1 = mass_balance_TM_elements_dfs[reactor_names[0]]["Time Point"]
            time_points_reactor_2 = mass_balance_TM_elements_dfs[reactor_names[1]]["Time Point"]
            time_points_reactor_3 = mass_balance_TM_elements_dfs[reactor_names[2]]["Time Point"]

            values_reactor_1 = mass_balance_TM_elements_dfs[reactor_names[0]][f'{element} (SUP)']/mass_balance_TM_elements_dfs[reactor_names[0]][f'{element} (ACC)']*100
            values_reactor_2 = mass_balance_TM_elements_dfs[reactor_names[1]][f'{element} (SUP)']/mass_balance_TM_elements_dfs[reactor_names[1]][f'{element} (ACC)']*100
            values_reactor_3 = mass_balance_TM_elements_dfs[reactor_names[2]][f'{element} (SUP)']/mass_balance_TM_elements_dfs[reactor_names[2]][f'{element} (ACC)']*100
            
            average_time_points = np.mean([time_points_reactor_1, time_points_reactor_2, time_points_reactor_3], axis=0)
            average_values = np.mean([values_reactor_1, values_reactor_2, values_reactor_3], axis=0)
            std_devs = np.std([values_reactor_1, values_reactor_2, values_reactor_3], axis=0)
                
            ax.bar(bar_positions[1:-1], average_values[1:-1], 0.20, color='black', alpha=0.70)
            ax.errorbar(bar_positions[1:-1], average_values[1:-1], yerr=std_devs[1:-1], color='red', capsize=7, linestyle="")

            vertical_line_pos = (bar_positions[2] + bar_positions[3]) / 2
            ax.axvline(x=vertical_line_pos, color='black', linestyle='-', alpha=0.95, linewidth=0.55)
            
            ax.fill_between(bar_positions, 0, 100, where=(bar_positions < 39), color='black', alpha=0.030)
            
            ax.axhline(y=100, color='black', linestyle='--', alpha=0.55)
            ax.set_ylim(-10, 300)

            ax.set_xlabel('Time [h]')
            ax.set_ylabel('Percentage (%)')
            
            ax.axhline(y=0, color='black', linestyle='-', alpha=1, linewidth=0.65)

            reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"
                                                                    
            ax.set_title(f'Mass Balance (%): {element}\n{reactor_name_triplicate} | {used_strain} | {iron_zinc_condition}')

            ax.text(0.10, 0.95, 'Batch phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.80)
            ax.text(0.57, 0.95, 'Feed phase', transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black', alpha=0.80)

            ax.set_xticks(bar_positions)
            ax.set_xticklabels([f'{round(label, 1):.0f}' for label in average_time_points])
            
            ax.spines['bottom'].set_visible(True)

            for pos in bar_positions:
                ax.axvline(x=pos, color='gray', linestyle='--', linewidth=0.25)

            if IS_SAVE_FIGURES:
                plt.savefig(f'figures/{folder_name}/MB_TM_triplicate_{element}_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')
            
            plt.show()


#### functions for generating figures:

11-Elemental Growth Yield Slopes [g biomass] vs [g element]

12-Elemental Growth Yield Against Reference Values

In [11]:
def getAndDsiplayFinalSamplesYields():
    global final_sample_df_with_final_biomass_df
    global dry_weight_final_sample_df
    global organized_AMBR_data_dfs
    
    final_volumes = []

    for index, row in dry_weight_final_sample_df.iterrows():
        sample_name = row["Sample name"]
        if sample_name in organized_AMBR_data_dfs:
            liquid_volume = organized_AMBR_data_dfs[sample_name]["Liquid volume [L]"][8]
            final_volume = liquid_volume + 0.04
            final_volumes.append(final_volume)
        
    dry_weight_final_sample_df["final_volume [L]"] = final_volumes
    dry_weight_final_sample_df["Total dry biomass [g]"] = dry_weight_final_sample_df["Dry biomass [g]"] * (dry_weight_final_sample_df["final_volume [L]"]/0.040)

    final_sample_df = pd.DataFrame.from_dict({k: {list(dct.keys())[0]: list(dct.values())[0] for dct in data} for k, data in final_massic_concentration_per_element.items()}, orient='index')
    final_sample_df_with_final_biomass_df = pd.merge(dry_weight_final_sample_df, final_sample_df, left_on="Sample name", right_index=True)

    final_sample_df_with_final_biomass_df = final_sample_df_with_final_biomass_df.drop("Na [µg/g]", axis=1)
    final_sample_df_with_final_biomass_df = final_sample_df_with_final_biomass_df.drop("Ca [µg/g]", axis=1)
    display(final_sample_df_with_final_biomass_df)

def calculateAndPlotElementalGrowthYieldSlopes(number_of_reactors=12, folder_name=FOLDER_NAME_11):
    global elemental_growth_yields_per_reactor, mass_balance_TM_elements_dfs, TM_elements, mass_balance_data_per_elements, organized_AMBR_data_dfs
    
    for element in TM_elements:
        for i in range(0, number_of_reactors, 3):
            reactor_names = list(mass_balance_data_per_elements.keys())[i:i+3]

            bacth_medium, feed_medium = media_per_reactor[reactor_names[0]][0], media_per_reactor[reactor_names[0]][1]
            iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

            fig, ax = plt.subplots(figsize=(7*0.8, 5*0.8))
            ax.tick_params(axis='y', labelcolor='black')        
            ax.set_xlabel(f"Accumulated {element} in the biomass [g]")
            ax.set_ylabel("Formed biomass [gCDW]", color='black')

            colors = {"Fe": "blue", "Zn": "orange", "Mg": "red", "Mn": "green", "Cu": "magenta", "Co": "salmon", "K": "goldenrod", "Mo": "royalblue"}
            markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}

            # mmol (biomass) / µmol (TM) : need to be converted into: gDW (biomass)/g (TM)    
            MW_biomass = 1.00*getMolarMass("C") + 1.80*getMolarMass("H") + 0.50*getMolarMass("O") + 0.20*getMolarMass("N")
            
            x1 = mass_balance_TM_elements_dfs[reactor_names[0]][f'{element} (BIO)'][1:-1]*1e-6*getMolarMass(element)   # µmol -> mol -> [g]
            x2 = mass_balance_TM_elements_dfs[reactor_names[1]][f'{element} (BIO)'][1:-1]*1e-6*getMolarMass(element)   # µmol -> mol -> [g]  
            x3 = mass_balance_TM_elements_dfs[reactor_names[2]][f'{element} (BIO)'][1:-1]*1e-6*getMolarMass(element)   # µmol -> mol -> [g]
            y1 = organized_AMBR_data_dfs[reactor_names[0]]["Biomass [mmol]"][1:-1]*1e-3*MW_biomass                     # mmol -> mol -> [g]
            y2 = organized_AMBR_data_dfs[reactor_names[1]]["Biomass [mmol]"][1:-1]*1e-3*MW_biomass                     # mmol -> mol -> [g]
            y3 = organized_AMBR_data_dfs[reactor_names[2]]["Biomass [mmol]"][1:-1]*1e-3*MW_biomass                     # mmol -> mol -> [g]
                
            x = pd.concat([x1, x2, x3]).reset_index(drop=True)
            y = pd.concat([y1, y2, y3]).reset_index(drop=True)
                
            x_batch = pd.concat([x1[:2], x2[:2], x3[:2]]).reset_index(drop=True)
            y_batch = pd.concat([y1[:2], y2[:2], y3[:2]]).reset_index(drop=True)
                
            x_fed_batch = pd.concat([x1[2:], x2[2:], x3[2:]]).reset_index(drop=True)
            y_fed_batch = pd.concat([y1[2:], y2[2:], y3[2:]]).reset_index(drop=True)

            ax.scatter(x, y, marker=markers[element], color=colors[element], alpha=0.45*0.5)
            ax.scatter(x_fed_batch, y_fed_batch, marker=markers[element], color=colors[element], alpha=0.45)
            ax.scatter(x_batch, y_batch, marker=markers[element], color="grey", alpha=0.35)

            plt.axhline(y=0, color='black', linestyle='-', alpha=0.75)

            model = LinearRegression()
            model_fed_batch = LinearRegression()

            model.fit(np.array(x).reshape(-1, 1), np.array(y).reshape(-1, 1))
            model_fed_batch.fit(np.array(x_fed_batch).reshape(-1, 1), np.array(y_fed_batch).reshape(-1, 1))
            
            x_line = np.arange(-1*5,8.5*5, 0.01)
            
            ax.plot(x_line, model.predict(x_line.reshape(-1, 1)), color='black', linewidth=1.4, linestyle='-', alpha=0.35*0.75)
            ax.plot(x_line, model_fed_batch.predict(x_line.reshape(-1, 1)), color=colors[element], linewidth=1.4, linestyle='-', alpha=0.35*0.75)

            y_pred = model.predict(np.array(x).reshape(-1, 1))
            y_pred_fed_batch = model_fed_batch.predict(np.array(x_fed_batch).reshape(-1, 1))

            a = model.coef_[0][0]
            b = model.intercept_[0]
            equation = f'y = {a:.1f}x + {b:.1f}'
            r_squared = r2_score(y, y_pred)
            r_squared_text = f'R² = {r_squared:.2f}'
            
            ax.text(0.05, 0.90, "Batch + fed-batch: ", transform=ax.transAxes, fontsize=9, color="black", alpha=0.85)
            ax.text(0.05, 0.85, equation, transform=ax.transAxes, fontsize=9, color="black", alpha=0.75)
            ax.text(0.05, 0.80, r_squared_text, transform=ax.transAxes, fontsize=9, color="black", alpha=0.75)

            a_fed_batch = model_fed_batch.coef_[0][0]
            b_fed_batch = model_fed_batch.intercept_[0]
            equation_fed_batch = f'y = {a_fed_batch:.1f}x + {b_fed_batch:.1f}'
            r_squared_fed_batch = r2_score(y_fed_batch, y_pred_fed_batch)
            r_squared_text_fed_batch = f'R² = {r_squared_fed_batch:.2f}'
            
            ax.text(0.05, 0.725, "Fed-batch: ", transform=ax.transAxes, fontsize=9, color=colors[element], alpha=0.95)
            ax.text(0.05, 0.675, equation_fed_batch, transform=ax.transAxes, fontsize=9, color=colors[element], alpha=0.85)
            ax.text(0.05, 0.625, r_squared_text_fed_batch, transform=ax.transAxes, fontsize=9, color=colors[element], alpha=0.85)
        
            elemental_growth_yield = a                        # [gDW/g]
            elemental_growth_yield_fed_batch = a_fed_batch    # [gDW/g]

            reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"                                                

            if reactor_name_triplicate not in elemental_growth_yields_per_reactor:
                elemental_growth_yields_per_reactor[reactor_name_triplicate] = []

            elemental_growth_yields_per_reactor[reactor_name_triplicate].append({element: {
                "value": f"{round(elemental_growth_yield, 1)}", 
                "R^2": round(r_squared, 3), 
                "value_fed_batch": f"{round(elemental_growth_yield_fed_batch, 1)}", 
                "R^2_fed_batch": round(r_squared_fed_batch, 3), 
                "experimenatl": f"{round(0, 1)}"}})

            plt.ticklabel_format(axis='x', style='sci', scilimits=(0,0))

            xlim_values = {
                "Fe": (-2.5*1e-3, 3.0*1e-3), 
                "Zn": (-3.0*1e-4, 4.5*1e-4), 
                "Mg": (-1.0*1e-2, 3.0*1e-2), 
                "Mn":  (-6*1e-5, 8*1e-5), 
                "Cu": (-3*1e-5, 4*1e-5), 
                "Co": (-3*1e-5, 4*1e-5),
                "K": (-10*1e-2, 12*1e-2),
                "Mo": (-3*1e-5, 6*1e-5), 
                }
                
            ax.set_xlim(xlim_values[element])    
            ax.set_ylim(-1, 10)    

            plt.title(f"{element}  |  Elemental Growth Yield [gCDW/g]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}")

            if IS_SAVE_FIGURES:
                plt.savefig(f'figures/{folder_name}/elemental_growth_yield_{element}_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')

            plt.show()

def showElementalGrowthYieldTable():
    global elemental_growth_yields_per_reactor, elemental_growth_yields_per_reactor_df

    pd.set_option('display.precision', 2)
    pd.set_option('display.float_format', '{:.2f}'.format)

    reference_data = {'Fe': 200, 'Zn': 1e+4, 'Mg': 200, 'Mn': 1e+4, 'Cu': 1e+5, 'Co': 1e+5, 'K': 100, 'Mo': 1e+4}

    elemental_growth_yields_per_reactor_df = pd.DataFrame()

    for element in reference_data:
        elemental_growth_yields_per_reactor_df.loc[element, f'Reference'] = reference_data[element]

    for reactor_name, data_elements in elemental_growth_yields_per_reactor.items():
        for data_element in data_elements:
            for element, data in data_element.items():
                elemental_growth_yields_per_reactor_df.loc[element, f'{reactor_name}'] = data['value']

    for reactor_name, data_elements in elemental_growth_yields_per_reactor.items():
        for data_element in data_elements:
            for element, data in data_element.items():
                elemental_growth_yields_per_reactor_df.loc[element, f'{reactor_name} [fed_batch]'] = data['value_fed_batch']

    for reactor_name, data_elements in elemental_growth_yields_per_reactor.items():
        for data_element in data_elements:
            for element, data in data_element.items():
                elemental_growth_yields_per_reactor_df.loc[element, f'{reactor_name} (R^2)'] = data['R^2']

    for reactor_name, data_elements in elemental_growth_yields_per_reactor.items():
        for data_element in data_elements:
            for element, data in data_element.items():
                elemental_growth_yields_per_reactor_df.loc[element, f'{reactor_name} (R^2) [fed_batch]'] = data['R^2_fed_batch']

    display(elemental_growth_yields_per_reactor_df)
    pd.set_option('display.precision', 6)
    pd.set_option('display.float_format', '{:.6f}'.format)

def plotMeasuredElementalGrowthYieldsAgainstReferenceValuesPerRSquaredIndividual(folder_name=FOLDER_NAME_12):
    global elemental_growth_yields_per_reactor_df
    reactor_names = ['138_R01-02-03', '138_R04-05-06', '139_R06-07-08', '139_R09-10-11']
    colors = {"138_R01-02-03": "darkgreen", "138_R04-05-06": "lightgreen", "139_R06-07-08": "red", "139_R09-10-11": "lightcoral"}
    xlim_values = {"Fe": 5.80*10, "Zn": 10.90, "Mg": 5.30, "Mn": 1.20*100, "Cu": 1.1*10, "Co": 1.1*10, "K": 1.1*10, "Mo": 1.2*100}

    reference_values = {
        "Fe": r"$200$",
        "Zn": r"$10^4$",
        "Mg": r"$200$",
        "Mn": r"$10^4$",
        "Cu": r"$10^5$",
        "Co": r"$10^5$",
        "K": r"$100$",
        "Mo": r"$10^4$"}
    
    for element in elemental_growth_yields_per_reactor_df.index:
        
        fig, ax = plt.subplots(figsize=(4*0.925, 4*0.925))
        for reactor_name in reactor_names:
            
            x = float(elemental_growth_yields_per_reactor_df[f"{reactor_name}"][element])
            x_fed_batch = float(elemental_growth_yields_per_reactor_df[f"{reactor_name} [fed_batch]"][element])
                
            y = float(round(elemental_growth_yields_per_reactor_df[f"{reactor_name} (R^2)"][element], 2))
            y_fed_batch = float(round(elemental_growth_yields_per_reactor_df[f"{reactor_name} (R^2) [fed_batch]"][element], 2))
            
            # ax.scatter(x, y, marker='s', color="grey", alpha=0.8*0.5)
            # ax.scatter(x, y, marker='s', color=colors[reactor_name], alpha=0.5*0.45)           
                
            ax.scatter(x_fed_batch, y_fed_batch, marker='s', color=colors[reactor_name], alpha=0.55)

            
        reference_value = elemental_growth_yields_per_reactor_df['Reference'][element]
        ax.set_xlim([reference_value/ xlim_values[element], reference_value * xlim_values[element]])
        
        ax.axvline(x=reference_value, color='red', linestyle='--', alpha=0.55)

        ax.set_xlabel(f"Measured elemental growth yield [gCDW/g]")
        ax.set_ylabel("R²")
        ax.set_xscale("log")
        ax.set_ylim(0, 1)
        ax.grid(True)

        legend_elements = []
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["138_R01-02-03"], markersize=9, label="HMP3071 | ↑ Zn+Fe (fed-batch)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["138_R04-05-06"], markersize=9, label="HMP3071 | ↓ Zn+Fe (fed-batch)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["139_R06-07-08"], markersize=9, label="DDB35 | ↑ Zn+Fe (fed-batch)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["139_R09-10-11"], markersize=9, label="DDB35 | ↓ Zn+Fe (fed-batch)"))
        
        fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.005), ncol=2, frameon=False)

        plt.tight_layout()
        plt.title(f"\n{element} = {reference_values[element]}", fontsize=12)
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/measured_elemental_growth_yield_against_reference_values_individual_{element}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotMeasuredElementalGrowthYieldsAgainstReferenceValuesPerRSquaredSUBPLOT(folder_name=FOLDER_NAME_12):
    global elemental_growth_yields_per_reactor_df
    
    reactor_names = ['138_R01-02-03', '138_R04-05-06', '139_R06-07-08', '139_R09-10-11']
    colors = {"138_R01-02-03": "darkgreen", "138_R04-05-06": "lightgreen", "139_R06-07-08": "red", "139_R09-10-11": "lightcoral"}
    xlim_values = {"Fe": 5.80*10, "Zn": 10.90, "Mg": 5.30, "Mn": 1.20*100, "Cu": 1.1*10, "Co": 1.1*10, "K": 1.1*10, "Mo": 1.2*100}

    reference_values = {
        "Fe": r"$200$",
        "Zn": r"$10^4$",
        "Mg": r"$200$",
        "Mn": r"$10^4$",
        "Cu": r"$10^5$",
        "Co": r"$10^5$",
        "K": r"$100$",
        "Mo": r"$10^4$"}

    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 8))
    fig.suptitle("\n\nMeasured Elemental Growth Yield [gCDW/g] Vs. R²", y=1.02)

    for i, element in enumerate(elemental_growth_yields_per_reactor_df.index):
        ax = axes[i // 4, i % 4]
        for reactor_name in reactor_names:
            x = float(elemental_growth_yields_per_reactor_df[f"{reactor_name}"][element])
            y = float(round(elemental_growth_yields_per_reactor_df[f"{reactor_name} (R^2)"][element], 2))
            
            # ax.scatter(x, y, marker='s', color="grey", alpha=0.8*0.5)
            # ax.scatter(x, y, marker='s', color=colors[reactor_name], alpha=0.5*0.45)           
            
            x_fed_batch = float(elemental_growth_yields_per_reactor_df[f"{reactor_name} [fed_batch]"][element])                
            y_fed_batch = float(round(elemental_growth_yields_per_reactor_df[f"{reactor_name} (R^2) [fed_batch]"][element], 2))
            ax.scatter(x_fed_batch, y_fed_batch, marker='s', color=colors[reactor_name], alpha=0.55)

        reference_value = elemental_growth_yields_per_reactor_df['Reference'][element]
        ax.set_xlim([reference_value / xlim_values[element], reference_value * xlim_values[element]])

        ax.axvline(x=reference_value, color='red', linestyle='--', alpha=0.60)
        ax.set_xlabel(f"[gCDW/g]")
        ax.set_ylabel("R²")
        ax.set_xscale("log")
        ax.set_ylim(0, 1)
        ax.set_title(element)
        ax.set_title(f"{element} = {reference_values[element]}", fontsize=11)
        ax.grid(True)

    legend_elements = []
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["138_R01-02-03"], markersize=9, label="HMP3071 | ↑ Zn+Fe (fed-batch)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["138_R04-05-06"], markersize=9, label="HMP3071 | ↓ Zn+Fe (fed-batch)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["139_R06-07-08"], markersize=9, label="DDB35 | ↑ Zn+Fe (fed-batch)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["139_R09-10-11"], markersize=9, label="DDB35 | ↓ Zn+Fe (fed-batch)"))
    
    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.005), ncol=2, frameon=False)

    plt.tight_layout(rect=[0, 0, 1, 0.97])
    
    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/measured_elemental_growth_yields_against_reference_values_subplot.png', dpi=300, bbox_inches='tight')

    plt.show()

def plotMeasuredElementalGrowthYieldsAgainstReferenceValuesHorizontalPlot(folder_name=FOLDER_NAME_12):
    
    global elemental_growth_yields_per_reactor_df

    reactor_names = ['138_R01-02-03', '138_R04-05-06', '139_R06-07-08', '139_R09-10-11']
    colors = {"138_R01-02-03": "darkgreen", "138_R04-05-06": "lightgreen", "139_R06-07-08": "red", "139_R09-10-11": "lightcoral"}
    
    xlim_values = {"Fe": 5.80*10, "Zn": 10.90, "Mg": 5.30, "Mn": 1.20*100, "Cu": 1.1*10, "Co": 1.1*10, "K": 1.1*10, "Mo": 1.2*100}

    reference_values = {
        "Fe": r"$200$",
        "Zn": r"$10^4$",
        "Mg": r"$200$",
        "Mn": r"$10^4$",
        "Cu": r"$10^5$",
        "Co": r"$10^5$",
        "K": r"$100$",
        "Mo": r"$10^4$"}

    fig, axes = plt.subplots(nrows=8, ncols=1, figsize=(7.5*0.8, 8.0*0.9), sharex=False)

    for i, element in enumerate(elemental_growth_yields_per_reactor_df.index):
        ax = axes[i]
        ax.axvline(x=elemental_growth_yields_per_reactor_df['Reference'][element], color='red', linestyle='-', alpha=0.75)

        for reactor_name in reactor_names:
            ax.axhline(y=1.00, color='black', linestyle='-', alpha=0.025)

            x = float(elemental_growth_yields_per_reactor_df[f"{reactor_name}"][element])
            x_fed_batch = float(elemental_growth_yields_per_reactor_df[f"{reactor_name} [fed_batch]"][element])
                        
            # ax.scatter(x, 1.00, marker='s', color="grey", alpha=0.8*0.5)
            # ax.scatter(x, 1.00, marker='s', color=colors[reactor_name], alpha=0.5*0.45)           
            
            ax.scatter(x_fed_batch, 1.00, marker='s', color=colors[reactor_name], alpha=0.55)

        ax.set_title(f"{element} = {reference_values[element]}", fontsize=11)
        ax.set_xscale('log')  
        ax.set_yticks([])
        ax.grid(True)

        reference_value = elemental_growth_yields_per_reactor_df['Reference'].iloc[i]
        axes[i].set_xlim([reference_value/ xlim_values[element], reference_value * xlim_values[element]])
    
    legend_elements = []
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["138_R01-02-03"], markersize=9, label="HMP3071 | ↑ Zn+Fe (fed-batch)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["138_R04-05-06"], markersize=9, label="HMP3071 | ↓ Zn+Fe (fed-batch)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["139_R06-07-08"], markersize=9, label="DDB35 | ↑ Zn+Fe (fed-batch)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=colors["139_R09-10-11"], markersize=9, label="DDB35 | ↓ Zn+Fe (fed-batch)"))
        
    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.005), ncol=2, frameon=False)

    plt.tight_layout()

    plt.suptitle("\nMeasured elemental growth yield [gCDW/g]\nagainst reference values", y=1.1)
    
    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/measured_elemental_growth_yield_against_reference_values_horizontal.png', dpi=300, bbox_inches='tight')

    plt.show()


#### functions for generating figures:

13-Yield Slopes [µg element] vs [g biomass]

14-Yield Bar Plots [µg element.g biomass-1] vs sampling event

15-Yield Bar Plot [avg. fed-batch µg element.g biomass-1] vs per medium-condition per strain-type

In [12]:
def getCalculatedAndMeasuredYields(number_of_reactors=12):
    global TM_elements, mass_balance_data_per_elements, organized_AMBR_data_dfs, mass_balance_TM_elements_dfs, final_sample_df_with_final_biomass_df
    global measured_and_calculated_yields_df

    yields_data = []

    for element in TM_elements:
        for i in range(0, number_of_reactors, 3):

            reactor_names = list(mass_balance_data_per_elements.keys())[i:i+3]            
            reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split('_')[1].replace('R', '')}-{reactor_names[2].split('_')[1].replace('R', '')}"                                                

            x_vals = [organized_AMBR_data_dfs[reactor_name]["Biomass [g]"][1:-1] for reactor_name in reactor_names]                                     # [g] biomass
            y_vals = [mass_balance_TM_elements_dfs[reactor_name][f'{element} (BIO)'][1:-1]*getMolarMass(element) for reactor_name in reactor_names]     # µmol * g * mol^-1 = [µg] TM 

            # linear regression applied to 7 fed-batch data points per reactor
            model_fits = [LinearRegression().fit(x_val[2:].values.reshape(-1, 1), y_val[2:].values.reshape(-1, 1)) for x_val, y_val in zip(x_vals, y_vals)]

            calculated_yield_avg = np.mean([model.coef_[0][0] for model in model_fits])
            calculated_yield_std = np.std([model.coef_[0][0] for model in model_fits])

            measured_yields = [final_sample_df_with_final_biomass_df.loc[final_sample_df_with_final_biomass_df['Sample name'] == reactor_name, f"{element} [µg/g]"].values[0] for reactor_name in reactor_names]
            measured_yields_avg = np.mean(measured_yields)
            measured_yields_std = np.std(measured_yields)

            yields_data.append(
                {
                "reactor name triplicate": reactor_name_triplicate,
                "elements": element,
                "calc. avg. yield [µg/g]": calculated_yield_avg,
                "calc. std. yield [µg/g]": calculated_yield_std,
                "meas. avg. yield [µg/g]": measured_yields_avg,
                "meas. std. yield [µg/g]": measured_yields_std
                })

    df_columns = ["reactor name triplicate", "elements", "calc. avg. yield [µg/g]", "calc. std. yield [µg/g]", "meas. avg. yield [µg/g]", "meas. std. yield [µg/g]"]
    measured_and_calculated_yields_df = pd.DataFrame(yields_data, columns=df_columns)
    display(measured_and_calculated_yields_df)

In [13]:
## 1. 
def plotYieldSlopes(number_of_reactors=12, folder_name=FOLDER_NAME_13):
    global mass_balance_TM_elements_dfs, final_sample_df_with_final_biomass_df, organized_AMBR_data_dfs, TM_elements, mass_balance_data_per_elements

    for element in TM_elements:
        for i in range(0, number_of_reactors, 3):
            reactor_names = list(mass_balance_data_per_elements.keys())[i:i+3]
            
            bacth_medium, feed_medium = media_per_reactor[reactor_names[0]][0], media_per_reactor[reactor_names[0]][1]
            iron_zinc_condition, used_strain = getTitleInformation(reactor_names[0], bacth_medium, feed_medium)

            fig, ax = plt.subplots(figsize=(7*0.75, 5*1.05))
            ax.tick_params(axis='y', labelcolor='black')        
            ax.set_xlabel("Formed biomass [g]", color='black')
            ax.set_ylabel(f"Accumulated {element} in the biomass [µg]")

            alpha_value = 0.75

            ylim_values = {
                "Co": (-25, 35), 
                "Cu": (-50, 85), 
                "Fe": (-2500, 6000), 
                "K" : (-100000, 200000), 
                "Mg": (-20000, 50000), 
                "Mn": (-25, 90), 
                "Mo": (-30, 100), 
                "Zn": (-275, 800)
                }
            colors = {"Fe": "blue", "Zn": "orange", "Mg": "red", "Mn": "green", "Cu": "magenta", "Co": "salmon", "K": "goldenrod", "Mo": "royalblue"}
            markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}
 
            # 3 arrays with 7 data points per each individual reactor (batch+fed-batch)
            x1_x2_x3_arrays = [organized_AMBR_data_dfs[reactor_name]["Biomass [g]"][1:-1] for reactor_name in reactor_names]                                                   # [g] biomass
            y1_y2_y3_arrays = [mass_balance_TM_elements_dfs[reactor_name][f'{element} (BIO)'][1:-1]*getMolarMass(element) for reactor_name in reactor_names]                   # µmol * g * mol^-1 = [µg] TM 

            # 3 arrays with 5 data points per each individual reactor (fed-batch)
            x1_x2_x3_fed_batch_arrays = [organized_AMBR_data_dfs[reactor_name]["Biomass [g]"][1:-1][2:] for reactor_name in reactor_names]                                     # [g] biomass
            y1_y2_y3_fed_batch_arrays = [mass_balance_TM_elements_dfs[reactor_name][f'{element} (BIO)'][1:-1][2:]*getMolarMass(element) for reactor_name in reactor_names]     # µmol * g * mol^-1 = [µg] TM 
            
            # 1 array with 7x3=21 data points for each reactor triplicate
            x_tricplicate_batch_and_fed_batch_array = pd.concat(x1_x2_x3_arrays).reset_index(drop=True)
            y_tricplicate_batch_and_fed_batch_array = pd.concat(y1_y2_y3_arrays).reset_index(drop=True)
            
            # 1 array with 5x3=15 data points for each reactor triplicate
            x_tricplicate_fed_batch_array = pd.concat(x1_x2_x3_fed_batch_arrays).reset_index(drop=True)
            y_tricplicate_fed_batch_array = pd.concat(y1_y2_y3_fed_batch_arrays).reset_index(drop=True)
            
            # 1 regression model PER reactor for fed-batch data point = 3 models in total
            model_fits_per_reactor_fed_batch = [LinearRegression().fit(x_val.values.reshape(-1, 1), y_val.values.reshape(-1, 1)) for x_val, y_val in zip(x1_x2_x3_fed_batch_arrays, y1_y2_y3_fed_batch_arrays)]
            a_per_reactor_fed_batch = [model.coef_[0][0] for model in model_fits_per_reactor_fed_batch]
            b_per_reactor_fed_batch = [model.intercept_[0] for model in model_fits_per_reactor_fed_batch]
            
            # 1 regression model PER REACTOR TRIPLICATE = 2 models in total
            model_triplicate_batch_and_fed_batch = LinearRegression().fit(np.array(x_tricplicate_batch_and_fed_batch_array).reshape(-1, 1), np.array(y_tricplicate_batch_and_fed_batch_array).reshape(-1, 1))
            a_model_triplicate_bacth_and_fed_batch = model_triplicate_batch_and_fed_batch.coef_[0][0]
            b_model_triplicate_bacth_and_fed_batch = model_triplicate_batch_and_fed_batch.intercept_[0]
            
            model_triplicate_fed_batch = LinearRegression().fit(np.array(x_tricplicate_fed_batch_array).reshape(-1, 1), np.array(y_tricplicate_fed_batch_array).reshape(-1, 1))
            a_model_triplicate_fed_batch = model_triplicate_fed_batch.coef_[0][0]
            b_model_triplicate_fed_batch = model_triplicate_fed_batch.intercept_[0]
            
            # Plotting calculated points
            ax.scatter(x1_x2_x3_arrays, y1_y2_y3_arrays, marker=markers[element], color=colors[element], alpha=0.55*0.50)                      # all points
            ax.scatter(x1_x2_x3_arrays[2:], y1_y2_y3_arrays[2:], marker=markers[element], color=colors[element], alpha=0.65*0.50)  # fed-batch points only
            ax.scatter(x1_x2_x3_arrays[:2], y1_y2_y3_arrays[:2], marker=markers[element], color="grey", alpha=0.50)                # batch points only
            
            # Plotting the biomass and the accumulated element quntity from the final sample
            measured_final_yields = np.array([final_sample_df_with_final_biomass_df.loc[final_sample_df_with_final_biomass_df['Sample name'] == reactor_name, f"{element} [µg/g]"].values[0] for reactor_name in reactor_names])
            final_biomasses = np.array([final_sample_df_with_final_biomass_df.loc[final_sample_df_with_final_biomass_df['Sample name'] == reactor_name, "Total dry biomass [g]"].values[0] for reactor_name in reactor_names])

            ax.scatter(final_biomasses, measured_final_yields*final_biomasses, marker="*", color="dimgrey", edgecolors="black", s=105, alpha=0.55)
            ax.scatter(final_biomasses, measured_final_yields*final_biomasses, marker="*", label= "Final samples", color=colors[element], s=105, alpha=0.25)

            ## Plotting the regression line (y=ax+b) for model_triplicate_batch_and_fed_batch + model_triplicate_fed_batch
            x_line = np.arange(-1*2.5, 10*2.5, 0.01)

            ax.plot(x_line, model_triplicate_batch_and_fed_batch.predict(x_line.reshape(-1, 1)), color='black', linewidth=1.4, linestyle='-', alpha=0.15)
            ax.plot(x_line, model_triplicate_fed_batch.predict(x_line.reshape(-1, 1)), color=colors[element], linewidth=1.4, linestyle='-', alpha=0.35)

            ## Plotting the equtions + R²
            # 1
            if b_model_triplicate_bacth_and_fed_batch<0:
                equation = f'y = {a_model_triplicate_bacth_and_fed_batch:.1f} x - {abs(b_model_triplicate_bacth_and_fed_batch):.1f}'
            else:
                equation = f'y = {a_model_triplicate_bacth_and_fed_batch:.1f} x + {b_model_triplicate_bacth_and_fed_batch:.1f}'
            
            y_pred_triplicate_batch_and_fed_batch = model_triplicate_batch_and_fed_batch.predict(np.array(x_tricplicate_batch_and_fed_batch_array).reshape(-1, 1))
            
            r_squared_text = f'R²={r2_score(y_tricplicate_batch_and_fed_batch_array, y_pred_triplicate_batch_and_fed_batch):.2f}'
            ax.text(0.05, 0.950, f"Batch + fed-batch: {equation} ({r_squared_text})", transform=ax.transAxes, fontsize=9, color="black", alpha=0.85)
            
            # 2
            if b_model_triplicate_fed_batch<0:
                equation_fed_batch = f'y = {a_model_triplicate_fed_batch:.1f} x - {abs(b_model_triplicate_fed_batch):.1f}'
            else:
                equation_fed_batch = f'y = {a_model_triplicate_fed_batch:.1f} x + {b_model_triplicate_fed_batch:.1f}'
            
            y_pred_triplicate_fed_batch = model_triplicate_fed_batch.predict(np.array(x_tricplicate_fed_batch_array).reshape(-1, 1))
            
            r_squared_text_fed_batch = f'R²={r2_score(y_tricplicate_fed_batch_array, y_pred_triplicate_fed_batch):.2f}'
            ax.text(0.05, 0.910, f"Fed-batch: {equation_fed_batch} ({r_squared_text_fed_batch})", transform=ax.transAxes, fontsize=9, color=colors[element], alpha=0.95)

            # 3
            calc_yields_avg = np.mean(a_per_reactor_fed_batch)
            calc_yields_std = np.std(a_per_reactor_fed_batch)
            calc_yields_per_reactor_text = f" {a_per_reactor_fed_batch[0]:.1f}, {a_per_reactor_fed_batch[1]:.1f}, {a_per_reactor_fed_batch[2]:.1f} "
            ax.text(0.05, 0.850, f"Calc. [µg/g]: {calc_yields_avg:.1f} ± {calc_yields_std:.1f} [{calc_yields_per_reactor_text}]", transform=ax.transAxes, fontsize=9, color="black", alpha=0.55)
        
            # 4
            meas_yields_avg = np.mean(measured_final_yields)
            meas_yields_std = np.std(measured_final_yields)
            meas_yields_per_reactor_text = f" {measured_final_yields[0]:.1f}, {measured_final_yields[1]:.1f}, {measured_final_yields[2]:.1f} "
            ax.text(0.05, 0.810, f"Meas. [µg/g]: {meas_yields_avg:.1f} ± {meas_yields_std:.1f} [{meas_yields_per_reactor_text}]", transform=ax.transAxes, fontsize=9, color="black", alpha=0.55)

            plt.axhline(y=0, color='black', linestyle='-', alpha=0.65)

            reactor_name_triplicate = f"{reactor_names[0]}-{reactor_names[1].split("_")[1].replace('R', "")}-{reactor_names[2].split("_")[1].replace('R', "")}"                                                
            
            ax.set_xlim(-1, 10.5)    
            ax.set_ylim(ylim_values[element])    

            plt.title(f"{element} yield [µg/g]\n{reactor_name_triplicate}  |  {used_strain}  |  {iron_zinc_condition}")
            plt.legend(loc='lower right')
            
            if IS_SAVE_FIGURES:
                plt.savefig(f'figures/{folder_name}/yield_slopes_µg_per_g_{element}_{reactor_name_triplicate}.png', dpi=300, bbox_inches='tight')

            plt.show()

## 2. 

def plotYieldsWithErrorsBarPlotIndividual(folder_name=FOLDER_NAME_14):
    global measured_and_calculated_yields_df

    colors_calculated = ['blue', 'orange', 'red', 'green']
    colors_measured = ['#7F7FFF', '#FFD1A4', '#FF7F7F', '#7FFF7F']

    bar_width = 0.30
    line_width = 1.0
    capsize = 7
    alpha_bar = 0.55
    alpha_error_bar = 0.37
    
    rgba_colors_calculated = [(*mcolors.to_rgba(color)[:-1], alpha_bar) for color in colors_calculated]
    rgba_colors_measured = [(*mcolors.to_rgba(color)[:-1], alpha_bar) for color in colors_measured]

    elements = measured_and_calculated_yields_df['elements'].unique()
    for element in elements:
        fig, ax = plt.subplots(figsize=(5, 5))

        reactors = measured_and_calculated_yields_df['reactor name triplicate'].unique()
    
        for i, reactor in enumerate(reactors):
            reactor_data = measured_and_calculated_yields_df[(measured_and_calculated_yields_df['elements'] == element) & (measured_and_calculated_yields_df['reactor name triplicate'] == reactor)]
            
            x = [i*1.5, i*1.5 + bar_width]
            
            calc_yield = reactor_data['calc. avg. yield [µg/g]'].values[0]
            meas_yield = reactor_data['meas. avg. yield [µg/g]'].values[0]

            calc_error = reactor_data['calc. std. yield [µg/g]'].values[0]
            meas_error = reactor_data['meas. std. yield [µg/g]'].values[0]

            ax.bar(x[0], calc_yield, 
                   yerr=calc_error, 
                   color=colors_calculated[i], 
                   width=bar_width, 
                   alpha=alpha_bar, 
                   capsize=capsize, 
                   error_kw={'ecolor': 'black', 'elinewidth': line_width*1.50, 'capthick': line_width, 'alpha': alpha_error_bar})
            
            ax.bar(x[1], meas_yield, 
                   yerr=meas_error, 
                   color=colors_measured[i], 
                   width=bar_width, 
                   alpha=alpha_bar, 
                   capsize=capsize, 
                   error_kw={'ecolor': 'black', 'elinewidth': line_width*1.50, 'capthick': line_width, 'alpha': alpha_error_bar})
        
        ax.set_title(f'Yield {element} [µg/g]')
        ax.set_ylabel('Yield [µg/g]')
        ax.grid(True, color='gray', linestyle='--', linewidth=0.25)
        ax.axhline(y=0, color='black', linestyle='-', alpha=0.45)
        ax.set_xticks([])
        ax.set_xticklabels([])

        legend_elements = []
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[0], markersize=9, label="HMP3071 | ↑ Zn+Fe (calc.)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[1], markersize=9, label="HMP3071 | ↓ Zn+Fe (calc.)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[2], markersize=9, label="DDB35 | ↑ Zn+Fe (calc.)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[3], markersize=9, label="DDB35 | ↓ Zn+Fe (calc.)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[0], markersize=9, label="HMP3071 | ↑ Zn+Fe (meas.)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[1], markersize=9, label="HMP3071 | ↓ Zn+Fe (meas.)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[2], markersize=9, label="DDB35 | ↑ Zn+Fe (meas.)"))
        legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[3], markersize=9, label="DDB35 | ↓ Zn+Fe (meas.)"))
        
        ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.025), ncol=2, frameon=False, fontsize="small")

        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/yields_µg_per_g_bar_plot_{element}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotYieldsWithErrorsBarPlotSubplot(folder_name=FOLDER_NAME_14):
    global measured_and_calculated_yields_df

    colors_calculated = ['blue', 'orange', 'red', 'green']
    colors_measured = ['#7F7FFF', '#FFD1A4', '#FF7F7F', '#7FFF7F']
    bar_width = 0.30
    line_width = 1.0
    capsize = 7
    alpha_bar = 0.55
    alpha_error_bar = 0.37
    
    rgba_colors_calculated = [(*mcolors.to_rgba(color)[:-1], alpha_bar) for color in colors_calculated]
    rgba_colors_measured = [(*mcolors.to_rgba(color)[:-1], alpha_bar) for color in colors_measured]

    elements = measured_and_calculated_yields_df['elements'].unique()

    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle('\nYield [µg/g] per Element', fontsize=16)

    for i, element in enumerate(elements):
        row, col = divmod(i, 4)
        ax = axes[row, col]

        reactors = measured_and_calculated_yields_df['reactor name triplicate'].unique()
        for j, reactor in enumerate(reactors):
            reactor_data = measured_and_calculated_yields_df[(measured_and_calculated_yields_df['elements'] == element) & (measured_and_calculated_yields_df['reactor name triplicate'] == reactor)]
            x = [j*1.5, j*1.5 + bar_width]

            calc_yield = reactor_data['calc. avg. yield [µg/g]'].values[0]
            meas_yield = reactor_data['meas. avg. yield [µg/g]'].values[0]

            calc_error = reactor_data['calc. std. yield [µg/g]'].values[0]
            meas_error = reactor_data['meas. std. yield [µg/g]'].values[0]

            ax.bar(x[0], calc_yield, yerr=calc_error, color=colors_calculated[j], width=bar_width, alpha=alpha_bar, capsize=capsize, error_kw={'ecolor': 'black', 'elinewidth': line_width*1.50, 'capthick': line_width, 'alpha': alpha_error_bar})
            ax.bar(x[1], meas_yield, yerr=meas_error, color=colors_measured[j], width=bar_width, alpha=alpha_bar, capsize=capsize, error_kw={'ecolor': 'black', 'elinewidth': line_width*1.50, 'capthick': line_width, 'alpha': alpha_error_bar})

            ax.set_title(f'{element}', fontsize=15)
            ax.set_ylabel('Yield [µg/g]')

            if i in [1, 2, 3, 5, 6, 7]:
                ax.set_ylabel("")        

            ax.grid(True, color='gray', linestyle='--', linewidth=0.25)
            ax.axhline(y=0, color='black', linestyle='-', alpha=0.45)
            ax.set_xticks([])
            ax.set_xticklabels([])

    legend_elements = []
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[0], markersize=9, label="HMP3071 | ↑ Zn+Fe (calc.)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[1], markersize=9, label="HMP3071 | ↓ Zn+Fe (calc.)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[2], markersize=9, label="DDB35 | ↑ Zn+Fe (calc.)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_calculated[3], markersize=9, label="DDB35 | ↓ Zn+Fe (calc.)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[0], markersize=9, label="HMP3071 | ↑ Zn+Fe (meas.)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[1], markersize=9, label="HMP3071 | ↓ Zn+Fe (meas.)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[2], markersize=9, label="DDB35 | ↑ Zn+Fe (meas.)"))
    legend_elements.append(Line2D([0], [0], marker='s', color='w', markerfacecolor=rgba_colors_measured[3], markersize=9, label="DDB35 | ↓ Zn+Fe (meas.)"))
    
    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, +0.085), ncol=2, frameon=False, fontsize="large")

    # plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/#_yields_µg_per_g_bar_plot_subplot.png', dpi=300, bbox_inches='tight')

    plt.show()

## 3. 
    
def plotYieldsPerStrainAndMediumTypesSubplotAllElements(folder_name=FOLDER_NAME_15):
    global measured_and_calculated_yields_df

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(7*1.00, 7*1.15))
    fig.suptitle('\nCalculated Vs. measured yields [µg/g]\nper strain and medium type', x=0.60, y=1.15, fontsize=14)

    colors = {"Fe": "blue", "Zn": "orange", "Mg": "red", "Mn": "green", "Cu": "magenta", "Co": "salmon", "K": "goldenrod", "Mo": "royalblue"}
    markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}

    reactor_names = measured_and_calculated_yields_df['reactor name triplicate'].unique()

    for idx, ax in enumerate(axes.flatten()):
        
        reactor_name_triplicate = reactor_names[idx]
        first_reactor = reactor_name_triplicate.split("-")[0]
        bacth_medium, feed_medium = media_per_reactor[first_reactor][0], media_per_reactor[first_reactor][1]
        iron_zinc_condition, used_strain = getTitleInformation(first_reactor, bacth_medium, feed_medium)

        filtered_df = measured_and_calculated_yields_df[measured_and_calculated_yields_df['reactor name triplicate'] == reactor_name_triplicate]

        all_avg_measured_yields = []
        all_avg_calculated_yields = []

        ax.tick_params(axis='y', labelcolor='black')        
        ax.set_xlabel("Measured yields [µg/g]")
        ax.set_ylabel("Calculated yields [µg/g]")

        if idx in [0, 1]:
            ax.set_xlabel("")

        if idx in [1, 3]:
            ax.set_ylabel("")        

        for element in filtered_df['elements'].unique():
            element_data = filtered_df[filtered_df['elements'] == element]

            avg_measured_yield = element_data['meas. avg. yield [µg/g]'].values[0]
            avg_calculated_yield = element_data['calc. avg. yield [µg/g]'].values[0]
            
            std_measured_yield = element_data['meas. std. yield [µg/g]'].values[0]
            std_calculated_yield = element_data['calc. std. yield [µg/g]'].values[0]

            ax.errorbar(
                avg_measured_yield, 
                avg_calculated_yield, 
                xerr=std_measured_yield, 
                yerr=std_calculated_yield, 
                fmt=markers[element], 
                color=colors[element], 
                alpha=0.45, 
                markersize=5.5,
                elinewidth=1.0*1.50, 
                # ecolor="black", 
                capsize=7, 
                capthick=1.0)

            all_avg_measured_yields.append(avg_measured_yield)
            all_avg_calculated_yields.append(avg_calculated_yield)

            
        global_min = min(min(all_avg_measured_yields), min(all_avg_calculated_yields))
        global_max = max(max(all_avg_measured_yields), max(all_avg_calculated_yields))

        ax.plot([global_min*0.01, global_max*100.0], [global_min*0.01, global_max*100.0], color='grey', linestyle='--', alpha=0.5)
        
        ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=7))
        ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=7))

        ax.axhline(y=0, color='black', linestyle='-', alpha=0.35)
        ax.axvline(x=0, color='black', linestyle='-', alpha=0.35)
        ax.grid(True, color='gray', linestyle='--', linewidth=0.20)

        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_xlim([max(global_min*0.95*0.5, 1e-3), global_max*3.5])
        ax.set_ylim([max(global_min*0.95*0.5, 1e-3), global_max*3.5])

        ax.set_title(f"{used_strain}  |  {iron_zinc_condition}")

    legend_elements = [Line2D([0], [0], marker=markers[element], color='w', markerfacecolor=color, markersize=9, label=element) for element, color in colors.items()]
    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.580, -0.005), ncol=4, frameon=False)
    
    plt.tight_layout(rect=[0, 0, 1.15, 1.15])

    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/#_yields_per_strain_and_medium_types_subplot_all_elements.png', dpi=300, bbox_inches='tight')

    plt.show()

def plotYieldsPerStrainAndMediumTypesScatterPlotPerElement(folder_name=FOLDER_NAME_15):
    global measured_and_calculated_yields_df

    alpha_bar = 0.50

    markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}
    colors = {'138_R01-02-03': 'blue', '138_R04-05-06': 'orange', '139_R06-07-08': 'red', '139_R09-10-11': 'green'}
    rgba_colors = {key: (*mcolors.to_rgba(value)[:-1], alpha_bar * 1.45) for key, value in colors.items()}
   
    elements = measured_and_calculated_yields_df['elements'].unique()
    reactor_names = measured_and_calculated_yields_df['reactor name triplicate'].unique()

    for element in elements:
        fig, ax = plt.subplots(figsize=(4.15, 4.15))
        global_min_per_element, global_max_per_element = float('inf'), float('-inf')
        legend_elements = []
        
        for i, reactor_name in enumerate(reactor_names):
            element_data = measured_and_calculated_yields_df[(measured_and_calculated_yields_df['elements'] == element) & (measured_and_calculated_yields_df['reactor name triplicate'] == reactor_name)]

            meas_min = element_data['meas. avg. yield [µg/g]'].min() - 0.5 * element_data['meas. std. yield [µg/g]'].min()
            meas_max = element_data['meas. avg. yield [µg/g]'].max() + 0.5 * element_data['meas. std. yield [µg/g]'].max()
            calc_min = element_data['calc. avg. yield [µg/g]'].min() - 0.5 * element_data['calc. std. yield [µg/g]'].min()
            calc_max = element_data['calc. avg. yield [µg/g]'].max() + 0.5 * element_data['calc. std. yield [µg/g]'].max()

            global_min_per_element = min(global_min_per_element, meas_min, calc_min)
            global_max_per_element = max(global_max_per_element, meas_max, calc_max)

            first_reactor = reactor_name.split("-")[0]
            bacth_medium, feed_medium = media_per_reactor[first_reactor][0], media_per_reactor[first_reactor][1]
            iron_zinc_condition, used_strain = getTitleInformation(first_reactor, bacth_medium, feed_medium)

            avg_measured_yield = element_data['meas. avg. yield [µg/g]'].values[0]
            avg_calculated_yield = element_data['calc. avg. yield [µg/g]'].values[0]
            std_measured_yield = element_data['meas. std. yield [µg/g]'].values[0]
            std_calculated_yield = element_data['calc. std. yield [µg/g]'].values[0]
          
            ax.errorbar(
                avg_measured_yield, 
                avg_calculated_yield, 
                xerr=std_measured_yield, 
                yerr=std_calculated_yield, 
                fmt=markers[element], 
                color=colors[reactor_name], 
                alpha=alpha_bar, 
                markersize=6.5,
                elinewidth=1.0*1.50,  
                capsize=7, 
                capthick=1.0)
            
            legend_elements.append(Line2D([0], [0], marker=markers[element], color='w', markerfacecolor=rgba_colors[reactor_name], markersize=6.5, label=f"{used_strain}  |  {iron_zinc_condition}"))

        factor = 5
        ax.plot([global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor], 
                [global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor], 
                color='grey', 
                linestyle='--', 
                alpha=0.5)
        
        factor = 0.55
        ax.set_xlim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)
        ax.set_ylim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)
        
        ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=7))
        ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=7))

        ax.axhline(y=0, color='black', linestyle='-', alpha=0.35)
        ax.axvline(x=0, color='black', linestyle='-', alpha=0.35)
        ax.grid(True, color='gray', linestyle='--', linewidth=0.20)

        ax.set_xlabel("Measured yields [µg/g]")
        ax.set_ylabel("Calculated yields [µg/g]")
        
        ax.set_title(f"\nCalculated Vs. measured yields [µg/g]\nper strain and medium type\n{element}", fontsize=12)
            
        fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, +0.005), ncol=2, frameon=False, fontsize="small")

        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/yields_per_strain_and_medium_types_scatter_plot_per_element_{element}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotYieldsPerStrainAndMediumTypesSubplotPerElement(folder_name=FOLDER_NAME_15):
    global measured_and_calculated_yields_df, media_per_reactor
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    
    fig.suptitle('\nCalculated yields [µg/g] Vs. measured yields [µg/g]\nper element', fontsize=16)

    alpha_bar = 0.50
    markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}
    colors = {'138_R01-02-03': 'blue', '138_R04-05-06': 'orange', '139_R06-07-08': 'red', '139_R09-10-11': 'green'}
    rgba_colors = {key: (*mcolors.to_rgba(value)[:-1], alpha_bar * 1.45) for key, value in colors.items()}

    elements = measured_and_calculated_yields_df['elements'].unique()
    reactor_names = measured_and_calculated_yields_df['reactor name triplicate'].unique()

    for idx, (element, ax) in enumerate(zip(elements, axes.flatten())):

        global_min_per_element, global_max_per_element = float('inf'), float('-inf')
        legend_elements = []

        for reactor_name in reactor_names:
            element_data = measured_and_calculated_yields_df[(measured_and_calculated_yields_df['elements'] == element) & (measured_and_calculated_yields_df['reactor name triplicate'] == reactor_name)]

            meas_min = element_data['meas. avg. yield [µg/g]'].min() - 0.5 * element_data['meas. std. yield [µg/g]'].min()
            meas_max = element_data['meas. avg. yield [µg/g]'].max() + 0.5 * element_data['meas. std. yield [µg/g]'].max()
            calc_min = element_data['calc. avg. yield [µg/g]'].min() - 0.5 * element_data['calc. std. yield [µg/g]'].min()
            calc_max = element_data['calc. avg. yield [µg/g]'].max() + 0.5 * element_data['calc. std. yield [µg/g]'].max()

            global_min_per_element = min(global_min_per_element, meas_min, calc_min)
            global_max_per_element = max(global_max_per_element, meas_max, calc_max)

            first_reactor = reactor_name.split("-")[0]
            bacth_medium, feed_medium = media_per_reactor[first_reactor][0], media_per_reactor[first_reactor][1]
            iron_zinc_condition, used_strain = getTitleInformation(first_reactor, bacth_medium, feed_medium)

            avg_measured_yield = element_data['meas. avg. yield [µg/g]'].values[0]
            avg_calculated_yield = element_data['calc. avg. yield [µg/g]'].values[0]
            std_measured_yield = element_data['meas. std. yield [µg/g]'].values[0]
            std_calculated_yield = element_data['calc. std. yield [µg/g]'].values[0]
          
            ax.errorbar(
                avg_measured_yield, 
                avg_calculated_yield, 
                xerr=std_measured_yield, 
                yerr=std_calculated_yield, 
                fmt=markers[element], 
                color=colors[reactor_name], 
                alpha=alpha_bar, 
                markersize=6.5,
                elinewidth=1.0*1.50,  
                capsize=7, 
                capthick=1.0)
            
            legend_elements.append(Line2D([0], [0], marker="s", color='w', markerfacecolor=rgba_colors[reactor_name], markersize=6.5, label=f"{used_strain}  |  {iron_zinc_condition}"))
            
        ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=7))
        ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=7))

        ax.axhline(y=0, color='black', linestyle='-', alpha=0.35)
        ax.axvline(x=0, color='black', linestyle='-', alpha=0.35)
        ax.grid(True, color='gray', linestyle='--', linewidth=0.20)

        factor = 5
        ax.plot(
            [global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor], 
            [global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor], 
            color='grey', 
            linestyle='--', 
            alpha=0.5)
        
        factor = 0.55
        ax.set_xlim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)
        ax.set_ylim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)
        ax.set_title(f"{element}", fontsize=16)

        ax.set_xlabel("Measured yields [µg/g]")
        ax.set_ylabel("Calculated yields [µg/g]")        
        
        if idx in [0, 1, 2, 3]:
            ax.set_xlabel("")

        if idx in [1, 2, 3, 5, 6, 7]:
            ax.set_ylabel("")        

    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, +0.045), ncol=2, frameon=False, fontsize="large")
    
    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/#_yields_per_strain_and_medium_types_subplot_per_element.png', dpi=300, bbox_inches='tight')

    plt.show()

## 4. 

def plotYieldsPerStrainTypeSubplotAllElements(folder_name=FOLDER_NAME_16):
    global measured_and_calculated_yields_df

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(7, 7*1.25))
    fig.suptitle('\nYields in strain HMP3071 Vs. DDB35 [µg/g]', x=0.60, y=1.05, fontsize=14)

    colors = {"Fe": "blue", "Zn": "orange", "Mg": "red", "Mn": "green", "Cu": "magenta", "Co": "salmon", "K": "goldenrod", "Mo": "royalblue"}
    markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}

    reactor_names = measured_and_calculated_yields_df['reactor name triplicate'].unique()
    df_groups = [measured_and_calculated_yields_df[measured_and_calculated_yields_df['reactor name triplicate'] == name] for name in reactor_names]

    for row, yield_type in enumerate(['measured', 'calculated']):
        
        yield_col_prefix = 'meas.' if yield_type == 'measured' else 'calc.'
        
        for ax_idx, (ax, high_df, low_df) in enumerate(zip(axes[row], df_groups[::2], df_groups[1::2])):
            ax.tick_params(axis='y', labelcolor='black')
            
            ax.set_xlabel(f"{yield_type.capitalize()} yields for DDB35 [µg/g]")
            ax.set_ylabel(f"{yield_type.capitalize()} yields for HMP3071 [µg/g]" if ax_idx == 0 else "")
            
            ax.xaxis.set_major_locator(MaxNLocator(nbins=7))
            ax.yaxis.set_major_locator(MaxNLocator(nbins=7))
            
            ax.axhline(y=1e0, color='black', linestyle='-', alpha=0.45)
            ax.axvline(x=1e0, color='black', linestyle='-', alpha=0.45)
            ax.grid(True, color='gray', linestyle='--', linewidth=0.20)
            
            ax.set_xscale("log")
            ax.set_yscale("log")

            elements = list(high_df["elements"])
            for i, element in enumerate(elements):

                ax.errorbar(
                    high_df[f"{yield_col_prefix} avg. yield [µg/g]"].iloc[i], 
                    low_df[f"{yield_col_prefix} avg. yield [µg/g]"].iloc[i], 
                    xerr=high_df[f"{yield_col_prefix} std. yield [µg/g]"].iloc[i], 
                    yerr=low_df[f"{yield_col_prefix} std. yield [µg/g]"].iloc[i], 
                    fmt=markers[element], 
                    color=colors[element], 
                    alpha=0.55, 
                    markersize=6.5,
                    elinewidth=1.0*1.50,  
                    capsize=7, 
                    capthick=1.0)

            min_val = min(min(high_df["meas. avg. yield [µg/g]"]), 
                          min(high_df["calc. avg. yield [µg/g]"]), 
                          min(low_df["meas. avg. yield [µg/g]"]), 
                          min(low_df["calc. avg. yield [µg/g]"]))
            
            max_val = max(max(high_df["meas. avg. yield [µg/g]"]), 
                          max(high_df["calc. avg. yield [µg/g]"]), 
                          max(low_df["meas. avg. yield [µg/g]"]), 
                          max(low_df["calc. avg. yield [µg/g]"]))

            ax.plot([min_val*0.01, max_val*100.0], [min_val*0.01, max_val*100.0], color='grey', linestyle='--', alpha=0.5)

            ax.set_xlim([max(min_val*0.95*0.35, 1e-2), max_val*6.5])
            ax.set_ylim([max(min_val*0.95*0.35, 1e-2), max_val*6.5])
            ax.set_title(f"↑ Zn+Fe" if ax_idx == 0 else f"↓ Zn+Fe")

    legend_elements = [Line2D([0], [0], marker=markers[element], color='w', markerfacecolor=color, markersize=9, label=element) for element, color in colors.items()]
    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.580, -0.005), ncol=4, frameon=False)
    
    plt.tight_layout(rect=[0, 0, 1.15, 1.05])
    
    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/#_yields_per_strain_type_subplot_all_elements.png', dpi=300, bbox_inches='tight')

    plt.show()

def plotYieldsPerStrainTypeScatterPlotPerElement(folder_name=FOLDER_NAME_16):
    global measured_and_calculated_yields_df

    colors = {'↑ Zn+Fe (calc.)':'blue', '↓ Zn+Fe (calc.)': 'orange','↑ Zn+Fe (meas.)': 'red', '↓ Zn+Fe (meas.)': 'green'}
    alpha_bar = 0.55
    rgba_colors = {key: (*mcolors.to_rgba(value)[:-1], alpha_bar * 1.45) for key, value in colors.items()}
    markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}

    reactor_names = measured_and_calculated_yields_df['reactor name triplicate'].unique()
    df_groups = [measured_and_calculated_yields_df[measured_and_calculated_yields_df['reactor name triplicate'] == name] for name in reactor_names]

    elements = measured_and_calculated_yields_df['elements'].unique()

    for element in elements:
        fig, ax = plt.subplots(figsize=(4.25, 4.25))

        global_min_per_element, global_max_per_element = float('inf'), float('-inf')

        ax.set_xlabel(f"Yields for DDB35 [µg/g]")
        ax.set_ylabel(f"Yields for HMP3071 [µg/g]")

        ax.xaxis.set_major_locator(MaxNLocator(nbins=7))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=7))

        for yield_type in ['measured', 'calculated']:
            yield_col_prefix = 'meas.' if yield_type == 'measured' else 'calc.'

            for ax_idx, (high_df, low_df) in enumerate(zip(df_groups[::2], df_groups[1::2])):
                element_data_high = high_df[high_df["elements"] == element]
                element_data_low = low_df[low_df["elements"] == element]
                
                meas_min_high = element_data_high['meas. avg. yield [µg/g]'].min() - 0.5 * element_data_high['meas. std. yield [µg/g]'].min()
                meas_max_high = element_data_high['meas. avg. yield [µg/g]'].max() + 0.5 * element_data_high['meas. std. yield [µg/g]'].max()
                calc_min_high = element_data_high['calc. avg. yield [µg/g]'].min() - 0.5 * element_data_high['calc. std. yield [µg/g]'].min()
                calc_max_high = element_data_high['calc. avg. yield [µg/g]'].max() + 0.5 * element_data_high['calc. std. yield [µg/g]'].max()

                meas_min_low = element_data_low['meas. avg. yield [µg/g]'].min() - 0.5 * element_data_low['meas. std. yield [µg/g]'].min()
                meas_max_low = element_data_low['meas. avg. yield [µg/g]'].max() + 0.5 * element_data_low['meas. std. yield [µg/g]'].max()
                calc_min_low = element_data_low['calc. avg. yield [µg/g]'].min() - 0.5 * element_data_low['calc. std. yield [µg/g]'].min()
                calc_max_low = element_data_low['calc. avg. yield [µg/g]'].max() + 0.5 * element_data_low['calc. std. yield [µg/g]'].max()

                global_min_per_element = min(global_min_per_element, meas_min_high, calc_min_high, meas_min_low, calc_min_low)
                global_max_per_element = max(global_max_per_element, meas_max_high, meas_max_low, calc_max_high, calc_max_low)

                color_key = f'{"↑" if ax_idx == 0 else "↓"} Zn+Fe ({yield_col_prefix})'
                
                ax.errorbar(
                    element_data_high[f"{yield_col_prefix} avg. yield [µg/g]"], 
                    element_data_low[f"{yield_col_prefix} avg. yield [µg/g]"], 
                    xerr=element_data_high[f"{yield_col_prefix} std. yield [µg/g]"], 
                    yerr=element_data_low[f"{yield_col_prefix} std. yield [µg/g]"], 
                    fmt=markers[element], 
                    color=colors[color_key], 
                    alpha=0.55, 
                    markersize=6.5,
                    elinewidth=1.0*1.50,  
                    capsize=7, 
                    capthick=1.0)

        
        factor = 5
        ax.plot([global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor], 
                [global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor], 
                color='grey', 
                linestyle='--', 
                alpha=0.5)
        
        factor = 0.45
        ax.set_xlim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)
        ax.set_ylim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)
        
        ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=7))
        ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=7))

        ax.axhline(y=0, color='black', linestyle='-', alpha=0.35)
        ax.axvline(x=0, color='black', linestyle='-', alpha=0.35)
        ax.grid(True, color='gray', linestyle='--', linewidth=0.20)
        
        legend_elements = []
        legend_elements.append(Line2D([0], [0], marker=markers[element], color='w', markerfacecolor=rgba_colors["↑ Zn+Fe (calc.)"], markersize=9, label="↑ Zn+Fe (calculated)"))
        legend_elements.append(Line2D([0], [0], marker=markers[element], color='w', markerfacecolor=rgba_colors["↓ Zn+Fe (calc.)"], markersize=9, label="↓ Zn+Fe (calculated)"))
        legend_elements.append(Line2D([0], [0], marker=markers[element], color='w', markerfacecolor=rgba_colors["↑ Zn+Fe (meas.)"], markersize=9, label="↑ Zn+Fe (measured)"))
        legend_elements.append(Line2D([0], [0], marker=markers[element], color='w', markerfacecolor=rgba_colors["↓ Zn+Fe (meas.)"], markersize=9, label="↓ Zn+Fe (measured)"))
        
        fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.005), ncol=2, frameon=False, fontsize="small")

        ax.set_title(f"\nYields in strain HMP3071 Vs. DDB35 [µg/g]\n{element}", fontsize=12)
        
        if IS_SAVE_FIGURES:
            plt.savefig(f'figures/{folder_name}/yields_per_strain_type_scatter_plot_per_element_{element}.png', dpi=300, bbox_inches='tight')

        plt.show()

def plotYieldsPerStrainTypeSubplotPerElement(folder_name=FOLDER_NAME_16):
    global measured_and_calculated_yields_df

    colors = {'↑ Zn+Fe (calc.)':'blue', '↓ Zn+Fe (calc.)': 'orange','↑ Zn+Fe (meas.)': 'red', '↓ Zn+Fe (meas.)': 'green'}
    alpha_bar = 0.55
    rgba_colors = {key: (*mcolors.to_rgba(value)[:-1], alpha_bar * 1.45) for key, value in colors.items()}
    markers = {"Fe": "s", "Zn": "D", "Mg": "P", "Mn": "v", "Cu": "p", "Co": "^", "K": "h", "Mo": "s"}

    reactor_names = measured_and_calculated_yields_df['reactor name triplicate'].unique()
    df_groups = [measured_and_calculated_yields_df[measured_and_calculated_yields_df['reactor name triplicate'] == name] for name in reactor_names]

    elements = measured_and_calculated_yields_df['elements'].unique()
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20*0.915, 10))
    
    fig.suptitle('\nYields in strain HMP3071 Vs. DDB35 [µg/g]\n', fontsize=16)

    for idx, element in enumerate(elements):
        ax = axes[idx // 4, idx % 4]
        global_min_per_element, global_max_per_element = float('inf'), float('-inf')

        ax.set_xlabel(f"Yields for DDB35 [µg/g]")
        ax.set_ylabel(f"Yields for HMP3071 [µg/g]")

        ax.xaxis.set_major_locator(MaxNLocator(nbins=7))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=7))

        for yield_type in ['measured', 'calculated']:
            yield_col_prefix = 'meas.' if yield_type == 'measured' else 'calc.'

            for ax_idx, (high_df, low_df) in enumerate(zip(df_groups[::2], df_groups[1::2])):
                element_data_high = high_df[high_df["elements"] == element]
                element_data_low = low_df[low_df["elements"] == element]

                meas_min_high = element_data_high['meas. avg. yield [µg/g]'].min() - 0.5 * element_data_high['meas. std. yield [µg/g]'].min()
                meas_max_high = element_data_high['meas. avg. yield [µg/g]'].max() + 0.5 * element_data_high['meas. std. yield [µg/g]'].max()
                calc_min_high = element_data_high['calc. avg. yield [µg/g]'].min() - 0.5 * element_data_high['calc. std. yield [µg/g]'].min()
                calc_max_high = element_data_high['calc. avg. yield [µg/g]'].max() + 0.5 * element_data_high['calc. std. yield [µg/g]'].max()

                meas_min_low = element_data_low['meas. avg. yield [µg/g]'].min() - 0.5 * element_data_low['meas. std. yield [µg/g]'].min()
                meas_max_low = element_data_low['meas. avg. yield [µg/g]'].max() + 0.5 * element_data_low['meas. std. yield [µg/g]'].max()
                calc_min_low = element_data_low['calc. avg. yield [µg/g]'].min() - 0.5 * element_data_low['calc. std. yield [µg/g]'].min()
                calc_max_low = element_data_low['calc. avg. yield [µg/g]'].max() + 0.5 * element_data_low['calc. std. yield [µg/g]'].max()

                global_min_per_element = min(global_min_per_element, meas_min_high, calc_min_high, meas_min_low, calc_min_low)
                global_max_per_element = max(global_max_per_element, meas_max_high, meas_max_low, calc_max_high, calc_max_low)

                color_key = f'{"↑" if ax_idx == 0 else "↓"} Zn+Fe ({yield_col_prefix})'

                ax.errorbar(
                    element_data_high[f"{yield_col_prefix} avg. yield [µg/g]"],
                    element_data_low[f"{yield_col_prefix} avg. yield [µg/g]"],
                    xerr=element_data_high[f"{yield_col_prefix} std. yield [µg/g]"],
                    yerr=element_data_low[f"{yield_col_prefix} std. yield [µg/g]"],
                    fmt=markers[element],
                    color=colors[color_key],
                    alpha=0.55,
                    markersize=6.5,
                    elinewidth=1.0*1.50,
                    capsize=7,
                    capthick=1.0)

        factor = 5
        ax.plot([global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor],
                [global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor],
                color='grey',
                linestyle='--',
                alpha=0.5)

        factor = 0.45
        ax.set_xlim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)
        ax.set_ylim(global_min_per_element-(global_max_per_element-global_min_per_element)*factor, global_max_per_element+(global_max_per_element-global_min_per_element)*factor)

        ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=7))
        ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=7))

        ax.axhline(y=0, color='black', linestyle='-', alpha=0.35)
        ax.axvline(x=0, color='black', linestyle='-', alpha=0.35)
        ax.grid(True, color='gray', linestyle='--', linewidth=0.20)
        
        ax.set_title(f"{element}", fontsize=16)
        
        if idx in [0, 1, 2, 3]:
            ax.set_xlabel("")

        if idx in [1, 2, 3, 5, 6, 7]:
            ax.set_ylabel("")

    legend_elements = []
    legend_elements.append(Line2D([0], [0], marker="s", color='w', markerfacecolor=rgba_colors["↑ Zn+Fe (calc.)"], markersize=9, label="↑ Zn+Fe (calculated)"))
    legend_elements.append(Line2D([0], [0], marker="s", color='w', markerfacecolor=rgba_colors["↓ Zn+Fe (calc.)"], markersize=9, label="↓ Zn+Fe (calculated)"))
    legend_elements.append(Line2D([0], [0], marker="s", color='w', markerfacecolor=rgba_colors["↑ Zn+Fe (meas.)"], markersize=9, label="↑ Zn+Fe (measured)"))
    legend_elements.append(Line2D([0], [0], marker="s", color='w', markerfacecolor=rgba_colors["↓ Zn+Fe (meas.)"], markersize=9, label="↓ Zn+Fe (measured)"))

    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, +0.005), ncol=2, frameon=False, fontsize="large")

    plt.tight_layout()
    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/#_yields_per_strain_type_subplot_per_element.png', dpi=300, bbox_inches='tight')

    plt.show()

## 5. 
    
def calculateChangeInPercentageOfYield():
    global measured_and_calculated_yields_df, percentage_df

    reactor_to_strain_and_medium_map = {
        "138_R01-02-03": "HMP3071 | ↑ Zn+Fe",
        "138_R04-05-06": "HMP3071 | ↓ Zn+Fe",
        "139_R06-07-08": "DDB35 | ↑ Zn+Fe",
        "139_R09-10-11": "DDB35 | ↓ Zn+Fe"
    }

    measured_and_calculated_yields_df['conditions'] = measured_and_calculated_yields_df['reactor name triplicate'].map(reactor_to_strain_and_medium_map)

    high_yield_df = measured_and_calculated_yields_df[measured_and_calculated_yields_df['conditions'].str.contains("↑")]
    low_yield_df = measured_and_calculated_yields_df[measured_and_calculated_yields_df['conditions'].str.contains("↓")]

    percentage_df = low_yield_df.copy()

    for index, row in low_yield_df.iterrows():
        element = row['elements']
        condition_high = row['conditions'].replace("↓", "↑")
        condition_low = row['conditions']
        
        new_condition = f"{condition_low.split("|")[0].strip()} ↑ Zn+Fe -> ↓ Zn+Fe"
        
        high_yield_row = high_yield_df[(high_yield_df['elements'] == element) & (high_yield_df['conditions'] == condition_high)]

        if not high_yield_row.empty:
            percentage_df.at[index, '% change in calc. yield'] = ((row['calc. avg. yield [µg/g]'] - high_yield_row.iloc[0]['calc. avg. yield [µg/g]']) / high_yield_row.iloc[0]['calc. avg. yield [µg/g]']) * 100
            percentage_df.at[index, '% change in meas. yield'] = ((row['meas. avg. yield [µg/g]'] - high_yield_row.iloc[0]['meas. avg. yield [µg/g]']) / high_yield_row.iloc[0]['meas. avg. yield [µg/g]']) * 100

        percentage_df.at[index, 'conditions'] = new_condition

    percentage_df = percentage_df[['conditions', 'elements', '% change in calc. yield', '% change in meas. yield']]
    display(percentage_df)

def getReshapedPercentageData():
    global percentage_df, reshaped_df
    
    reshaped_df = pd.DataFrame()

    for condition in ["HMP3071 ↑ Zn+Fe -> ↓ Zn+Fe", "DDB35 ↑ Zn+Fe -> ↓ Zn+Fe"]:
        for yield_type in ['% change in calc. yield', '% change in meas. yield']:
            temp_df = percentage_df[percentage_df['conditions'] == condition][['elements', yield_type]]
            temp_df.columns = ['elements', f'{condition} | {yield_type}']
            reshaped_df = pd.merge(reshaped_df, temp_df, on='elements', how='outer') if not reshaped_df.empty else temp_df

    reshaped_df = reshaped_df.set_index('elements')
    reshaped_df = reshaped_df.reindex(columns=sorted(reshaped_df.columns))
    reshaped_df.insert(2, '', np.nan)

    display(reshaped_df)

def plotChangeInYieldPerElementHeatmap(folder_name=FOLDER_NAME_17):
    global percentage_df, reshaped_df

    plt.figure(figsize=(10*1.10, 8*0.7))

    heatmap = sns.heatmap(
        reshaped_df, 
        annot=True, 
        cmap='RdYlGn', 
        center=0, vmin=-100, vmax=100, 
        fmt='.1f', 
        linewidths=1.25, 
        annot_kws={"size": 10, "color": "black", "weight": "regular"}, 
        alpha=0.65)

    for text in heatmap.texts:
        text.set_text(f'{text.get_text()} %')

    heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0)
    plt.yticks(rotation=0)

    plt.xticks([])
    plt.xticks([], [])

    plt.ylabel("")

    plt.text(1, -0.25, "DDB35", ha='center', va='center', fontsize=11)
    plt.text(4, -0.25, "HMP3071", ha='center', va='center', fontsize=11)

    for i, label in enumerate(["% in calc. yield", "% in meas. yield", "", "% in calc. yield", "% in meas. yield"]):
        plt.text(i + 0.5, len(reshaped_df) + 0.25, label, ha='center', va='center', fontsize=10)

    plt.text(2.50, -0.95, "\nThe effect of iron and zinc in the medium on the element's yield\nChange in the yield [%] [↑ Zn+Fe -> ↓ Zn+Fe]\n\n", ha='center', va='center', fontsize=11)
    
    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/change_in_yield_heatmap.png', dpi=300, bbox_inches='tight')

    plt.show()

## 6. 
    
def plotMeasuredYieldsHorizontal(folder_name=FOLDER_NAME_17):
    global measured_and_calculated_yields_df

    reactor_names_color_map = {
        "HMP3071 | high Zn+Fe": "blue", 
        "HMP3071 | low Zn+Fe": "orange", 
        "DDB35 | high Zn+Fe": "red", 
        "DDB35 | low Zn+Fe": "green"
    }

    reactor_to_condition_map = {
        "138_R01-02-03": "HMP3071 | high Zn+Fe",
        "138_R04-05-06": "HMP3071 | low Zn+Fe",
        "139_R06-07-08": "DDB35 | high Zn+Fe",
        "139_R09-10-11": "DDB35 | low Zn+Fe"
    }

    measured_and_calculated_yields_df['Condition'] = measured_and_calculated_yields_df['reactor name triplicate'].map(reactor_to_condition_map)

    avg_yield_pivot = measured_and_calculated_yields_df.pivot_table(index='elements', columns='Condition', values='meas. avg. yield [µg/g]')
    std_yield_pivot = measured_and_calculated_yields_df.pivot_table(index='elements', columns='Condition', values='meas. std. yield [µg/g]')

    elements = ["Zn", "Fe", "Mn", "Cu", "Co", "Mo", "Mg", "K"]
    
    fig, axes = plt.subplots(nrows=len(avg_yield_pivot), ncols=1, figsize=(7.5*0.8, 8.0*0.9), sharex=False)

    for i, element in enumerate(elements):
        ax = axes[i]
        ax.axhline(y=1.00, color='black', linestyle='-', alpha=0.025)

        for reactor_name in avg_yield_pivot.columns:
            x = avg_yield_pivot.loc[element, reactor_name]
            std_dev = std_yield_pivot.loc[element, reactor_name]

            if "HMP3071" in reactor_name:
                marker_per_strain = "^"
            else:
                marker_per_strain = "s"

            ax.errorbar(x, y=1.0, xerr=std_dev, fmt=marker_per_strain, color=reactor_names_color_map[reactor_name], alpha=0.55, capsize=3)
            ax.set_title(f"{element}", fontsize=11)

        ax.set_yticks([])
        ax.grid(True)

    strain_marker_map = {
        "HMP3071": "^", 
        "DDB35": "s"    
    }

    legend_elements = []
    for reactor_name, color in reactor_names_color_map.items():
        strain = reactor_name.split(" | ")[0]
        marker = strain_marker_map[strain]
        label = reactor_name.replace("high", "↑").replace("low", "↓")
        legend_elements.append(Line2D([0], [0], marker=marker, color='w', markerfacecolor=color, markersize=9, label=label))

    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.005), ncol=2, frameon=False)

    plt.tight_layout()
    plt.suptitle("\nMeasured yields [µg/g]", y=1.1)

    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/measured_yields_horizontal.png', dpi=300, bbox_inches='tight')

    plt.show()

def plotCalculatedYieldsHorizontal(folder_name=FOLDER_NAME_17):
    global measured_and_calculated_yields_df

    reactor_names_color_map = {
        "HMP3071 | high Zn+Fe": "blue", 
        "HMP3071 | low Zn+Fe": "orange", 
        "DDB35 | high Zn+Fe": "red", 
        "DDB35 | low Zn+Fe": "green"
    }

    reactor_to_condition_map = {
        "138_R01-02-03": "HMP3071 | high Zn+Fe",
        "138_R04-05-06": "HMP3071 | low Zn+Fe",
        "139_R06-07-08": "DDB35 | high Zn+Fe",
        "139_R09-10-11": "DDB35 | low Zn+Fe"
    }

    measured_and_calculated_yields_df['Condition'] = measured_and_calculated_yields_df['reactor name triplicate'].map(reactor_to_condition_map)

    avg_yield_pivot = measured_and_calculated_yields_df.pivot_table(index='elements', columns='Condition', values='calc. avg. yield [µg/g]')
    std_yield_pivot = measured_and_calculated_yields_df.pivot_table(index='elements', columns='Condition', values='calc. std. yield [µg/g]')

    elements = ["Zn", "Fe", "Mn", "Cu", "Co", "Mo", "Mg", "K"]

    fig, axes = plt.subplots(nrows=len(avg_yield_pivot), ncols=1, figsize=(7.5*0.8, 8.0*0.9), sharex=False)

    for i, element in enumerate(elements):
        ax = axes[i]
        ax.axhline(y=1.00, color='black', linestyle='-', alpha=0.025)

        for reactor_name in avg_yield_pivot.columns:
            x = avg_yield_pivot.loc[element, reactor_name]
            std_dev = std_yield_pivot.loc[element, reactor_name]

            if "HMP3071" in reactor_name:
                marker_per_strain = "^"
            else:
                marker_per_strain = "s"

            ax.errorbar(x, y=1.0, xerr=std_dev, fmt=marker_per_strain, color=reactor_names_color_map[reactor_name], alpha=0.55, capsize=3)
            ax.set_title(f"{element}", fontsize=11)

        ax.set_yticks([])
        ax.grid(True)

    strain_marker_map = {
        "HMP3071": "^", 
        "DDB35": "s"    
    }

    legend_elements = []
    for reactor_name, color in reactor_names_color_map.items():
        strain = reactor_name.split(" | ")[0]
        marker = strain_marker_map[strain]
        label = reactor_name.replace("high", "↑").replace("low", "↓")
        legend_elements.append(Line2D([0], [0], marker=marker, color='w', markerfacecolor=color, markersize=9, label=label))

    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.005), ncol=2, frameon=False)

    plt.tight_layout()
    plt.suptitle("\nCalculated yields [µg/g]", y=1.1)

    if IS_SAVE_FIGURES:
        plt.savefig(f'figures/{folder_name}/calculated_yields_horizontal.png', dpi=300, bbox_inches='tight')

    plt.show()

